##0 Setup

In [ ]:
!pip install latextable
!pip install names
!pip install num2words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for latextable: filename=latextable-0.3.0-py3-none-any.whl size=7254 sha256=4cac4d94dc67ea5a4023d69c32ca2c9c8402602f95ecae6571c52d49dc103f03
  Stored in directory: /root/.cache/pip/wheels/a3/8d/a3/41e5c8fb7490c6e93ceb64ca1740f172b7ea837fe42ac41ce0
Successfully built latextable
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 789 kB 35.9 MB/s 
  Created wheel for names: filename=names-0.3.0-py3-none-any.whl size=803699 sha256=9bc3cf2215401a862fbc27519f3ff94be89fe715aa4a1ac8d01cf808e0a0bb2b
  Stored in directory: /root/.cache/pip/wheels/05/ea/68/92f6b0669e478af9b7c3c524520d03050089e034edcc775c2b
Successfully built names
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 125 kB 21.0 MB/s 
  Created wh

In [ ]:
import random
from random import randint, choice, sample

import latextable
from texttable import Texttable

from sympy import *

variables = 'a b c d m n p q r s t w x y'
variables = symbols(variables, real=True)
variables += (Symbol('z'),)
a,b,c,d,m,n,p,q,r,s,t,w,x,y,z = variables

###Functions and sequences

In [ ]:
class LinFunc():
    """
    Represent a linear function.

    Attributes
    ----------
        slope : number (int, float, or Rational)
        intercept : number
        variable : sympy symbol, default=x
        label : str, default='f'
        expr : sympy expression
        domain : sympy set, default=S.Reals
        precision : int, default=2
    """

    def __init__(self, slope, pt, variable=x, label='f', domain=S.Reals, precision=2):
        """
        Construct linear function object from slope and y-intercept.

        Parameters
        ----------
            slope : number (int, float, or Rational)
            pt : number or list
            variable : sympy symbol, default=x
            label : str, default='f'
            domain : sympy set, default=S.Reals
            precision : int, default=2
        """
        
        self.slope, pt = sympify([slope, pt])
        
        self.intercept = pt[1] - pt[0]*self.slope if type(pt)==list else pt
        self.expr = self.slope*variable + self.intercept

        self.variable = variable
        self.label = label
        self.domain = domain
        self.precision = precision
    
    def __str__(self):
        """Return LaTeX formatted slope-intercept form."""
        
        if self.slope in [0,1]:
            slopeInt = ''
        elif self.slope == -1:
            slopeInt = '-'
        else:
            slopeInt = latexify(self.slope, self.precision)
        slopeInt += latex(self.variable) if self.slope!=0 else '' 
        
        intercept = latexify(self.intercept, self.precision)
        if self.intercept>0 and self.slope!=0:
            slopeInt += ' + '
        if self.intercept!=0 or self.slope==0:
            slopeInt += intercept
            
        return slopeInt
    
    @classmethod
    def fromPts(cls, pt1, pt2, variable=x, label='f', domain=S.Reals, precision=2):
        """
        Construct linear function object from two ordered pairs.

        Parameters
        ----------
            pt1, pt2 : list or tuple
            variable : sympy symbol, default=x
            label : str, default='f'
        """

        pt1, pt2 = sympify([pt1, pt2])
        
        slope = (pt2[1] - pt1[1])/(pt2[0] - pt1[0])
        
        if pt1[0]==0:
            return cls(slope, pt1[1], variable, label, domain, precision)
        elif pt2[0]==0:
            return cls(slope, pt2[1], variable, label, domain, precision)
        else:
            return cls(slope, pt1, variable, label, domain, precision)

    def subs(self, input):
        """Return evaluation of function at input."""

        return self.expr.subs(self.variable, sympify(input))

    def subSet(self, preimage):
        """Return image of set."""

        return ImageSet(Lambda(self.variable, self.expr), preimage).intersect(S.Reals)

    def getRange(self):
        """Return range of function."""

        return self.subSet(self.domain)
        
    def solve(self, output):
        if self.slope == 0:
            return self.domain if output==self.intercept else None
        else:
            return (sympify(output)-self.intercept)/self.slope

    def getSlopeInt(self, notation='func'):
        """Return LaTeX formatted slope-intercept form."""

        if notation == 'func':
            return f'{self.label}({self.variable}) = ' + str(self)
        else:
            return notation + '=' + str(self)

    def getPtSlope(self, pt=1, notation='func'):
        """
        Return LaTeX formatted point-slope form.
        
        Parameters
        ---------
            pt : number or list/tuple
                Independent variable value or ordered pair.
            notation : string, default='func'
                LHS of equality, default returns function notation.
        """

        pt = sympify(pt)
        if type(pt)!=list:
            pt = [pt] + [self.subs(pt)]
        
        ptTex = latexify(pt, self.precision)
        
        if self.slope in [0,1]:
            ptSlope = ''
        elif self.slope == -1:
            ptSlope = '-'
        else:
            ptSlope = latexify(self.slope, self.precision)
            
        if self.slope!=0:
            ptSlope += r'\left(' + latex(self.variable)
            ptSlope += '+' + ptTex[0][1:] if pt[0]<0 else '-' + ptTex[0]
            ptSlope += r'\right)'
        
        if pt[1]>0 and self.slope!=0:
            ptSlope += ' + '
        if pt[1]!=0 or self.slope==0:
            ptSlope += ptTex[1]
        
        if notation == 'func':
            return fr'{self.label}({self.variable}) = ' + ptSlope
        else:
            return notation + '=' + ptSlope
            
    def getStdForm(self, notation='y', mult=1):
        if self.slope == 0:
            mult = nsimplify(self.intercept).q
        elif self.intercept == 0:
            mult = nsimplify(self.slope).q
        else:
            mult *= abs(lcm(nsimplify(self.slope).q, nsimplify(self.intercept).q))
        mult = -mult if self.slope<=0 else mult
        
        stdForm = latex(nsimplify(mult*self.slope*self.variable - mult)) + notation
        stdForm += '=' + latex(nsimplify(-mult*self.intercept))
        
        return stdForm
        
    def getTable(self, nums, vals=True, vertical=True, labels=None):
        """
        Return LaTeX formatted table at given positions with or without blanks.
        
        Parameters
        ----------
            nums : list of ints
            vals : bool or list of bools or nonempty/empty strings ('')
            vertical : bool, default=True
            labels : list, default=None
                Defaults are independent variable and function notation.
        """
        
        vals = len(nums)*[vals] if type(vals)==bool else vals
        images = [signify(latexify(self.subs(jj), self.precision)) for jj in nums]
        images = [jj if vals[count] else r'\phantom{' + jj + r'}' for count,jj in enumerate(images)]
        
        nums = sympify(nums)
        nums = [signify(latexify(jj, self.precision)) for jj in nums]
        
        if labels == None:
            labels = [signify(str(self.variable)), fr'${self.label}({self.variable})$']
        if vertical:
            header = labels
            rows = list(zip(nums,images))
        else:
            header = [labels[0]] + list(nums)
            rows = [[labels[1]] + images]
        
        return tableGenerator(header, rows)

class ExpFunc():
    """
    Represent an (unshifted) exponential function.

    Attributes
    ----------
        base : number (int, float, or Rational)
        intercept : number
        variable : sympy symbol, default=x
        label : str, default='f'
        expr : sympy expression
        domain : sympy set, default=S.Reals
        precision : int, default=2
    """

    def __init__(self, base, pt, variable=x, label='f', domain=S.Reals, precision=2):
        """
        Construct exponential function object from base and y-intercept.

        Parameters
        ----------
            base : number (int, float, or Rational)
            intercept : number
            variable : sympy symbol, default=x
            label : str, default='f'
            domain : sympy set, default=S.Reals
            precision : int, default=2
        """

        self.base, pt = sympify([base, pt])
        
        self.intercept = pt[1]/Pow(self.base,pt[0]) if type(pt)==list else pt
        self.expr = self.intercept*(self.base**variable)
        
        self.variable = variable
        self.label = label
        self.domain = domain
        self.precision = precision
    
    def __str__(self):
        """Return LaTeX formatted base-intercept form."""

        if self.intercept == 1:
            baseInt = ''
        elif self.intercept == -1:
            baseInt = '-'
        else:
            baseInt = latexify(self.intercept, self.precision)
            if type(self.base)==Integer and self.base>0:
                baseInt += r' \cdot '
            
        if type(self.base)==Float or abs(self.base.q)>999:
            base = latexify(self.base, self.precision)
            baseInt += fr'({base})^{self.variable}'
        else:            
            baseInt += latex(self.base**self.variable)
            
        return baseInt
    
    @classmethod
    def fromPts(cls, pt1, pt2, variable=x, label='f', domain=S.Reals, precision=2):
        """
        Construct exponential function object from two ordered pairs.

        Parameters
        ----------
            pt1, pt2 : list or tuple
            variable : sympy symbol, default=x
            label : str, default='f'
        """
        
        pt1, pt2 = sympify([pt1, pt2])
        
        base = Pow(pt2[1]/pt1[1], 1/(pt2[0]-pt1[0]))
        
        if pt1[0]==0:
            return cls(base, pt1[1], variable, label, domain, precision)
        elif pt2[0]==0:
            return cls(base, pt2[1], variable, label, domain, precision)
        else:
            return cls(base, pt1, variable, label, domain, precision)

    def subs(self, input):
        """Return evaluation of function at input."""

        return self.expr.subs(self.variable, sympify(input))

    def subSet(self, preimage):
        """Return image of set."""

        return ImageSet(Lambda(self.variable, self.expr), preimage).intersect(S.Reals)

    def getRange(self):
        """Return range of function."""

        return self.subSet(self.domain)
        
    def solve(self, output):
        """Return preimage of given output."""
        
        return log(sympify(output)/self.intercept, self.base)
    
    def getBaseInt(self, notation='func'):
        """Return LaTeX formatted base-intercept form."""

        if notation == 'func':
            return fr'{self.label}({self.variable}) = ' + str(self)
        else:
            return notation + '=' + str(self)

    def getPtBase(self, pt=1, notation='func'):
        """
        Return LaTeX formatted point-base form.
        
        Parameter
        ---------
            pt : number or list/tuple
                Independent variable value or ordered pair.
            notation : string, default='func'
                LHS of equality, default returns function notation.
        """

        pt = sympify(pt)
        if type(pt)!=list:
            pt = [pt] + [self.subs(pt)]
         
        if pt[1] == 1:
            ptBase = ''
        elif pt[1] == -1:
            ptBase = '-'
        else:
            ptBase = latexify(pt[1], self.precision)
            if type(self.base)==Integer and self.base>0:
                ptBase += r' \cdot '
            
        if type(self.base)==Float or abs(self.base.q)>999:
            base = latexify(self.base, self.precision)
            ptBase += fr'({base})^' + r'{' + latex(self.variable-pt[0]) + r'}'
        else:
            ptBase += latex(self.base**(self.variable - pt[0]))
            
        if notation == 'func':
            return fr'{self.label}({self.variable}) = ' + ptBase
        else:
            return notation + '=' + ptBase
        
    def getTable(self, nums, vals=True, vertical=True, labels=None):
        """
        Return LaTeX formatted table at given positions with or without blanks.
        
        Parameters
        ----------
            nums : list of ints
            vals : bool or list of bools or nonempty/empty strings ('')
            vertical : bool, default=True
            labels : list, default=None
                Defaults are independent variable and function notation.
        """
        
        vals = len(nums)*[vals] if type(vals)==bool else vals
        images = [signify(latexify(self.subs(jj), self.precision)) for jj in nums]
        images = [jj if vals[count] else r'\phantom{' + jj + r'}' for count,jj in enumerate(images)]
        
        nums = sympify(nums)
        nums = [signify(latexify(jj, self.precision)) for jj in nums]
        
        if labels == None:
            labels = [signify(str(self.variable)), fr'${self.label}({self.variable})$']
        if vertical:
            header = labels
            rows = list(zip(nums,images))
        else:
            header = [labels[0]] + list(nums)
            rows = [[labels[1]] + images]
        
        return tableGenerator(header, rows)

In [ ]:
class QuadFunc():
    """
    Represent a quadratic function.

    Attributes
    ----------
        aa : number (int, float, or Rational)
        bb : number
        cc : number
        
        variable : sympy symbol, default=x
        label : str, default='f'
        expr : sympy expression
        domain : sympy set, default=S.Reals
        precision : int, default=2
    """

    def __init__(self, aa, bb, cc, variable=x, label='f', domain=S.Reals, precision=2):
        """
        Construct quadratic function object from coefficients.

        Parameters
        ----------
            aa : number (int, float, or Rational)
            bb : number
            cc : number
            variable : sympy symbol, default=x
            label : str, default='f'
            domain : sympy set, default=S.Reals
            precision : int, default=2
        """
        
        self.aa, self.bb, self.cc = sympify([aa, bb, cc])
        
        self.expr = self.aa*variable**2 + self.bb*variable + self.cc

        self.variable = variable
        self.label = label
        self.domain = domain
        self.precision = precision
    
    def __str__(self):
        """Return LaTeX formatted standard form."""
        
        if self.aa in [0,1]:
            stdForm = ''
        elif self.aa == -1:
            stdForm = '-'
        else:
            stdForm = latexify(self.aa, self.precision)
        stdForm += latex(self.variable**2) if self.aa!=0 else '' 
        
        bb = latexify(self.bb, self.precision)
        if self.bb>0 and self.aa!=0:
            stdForm += ' + '
        if self.bb == -1:
            stdForm += ' - '
        elif self.bb not in [0,1]:
            stdForm += bb
        stdForm += latex(self.variable) if self.bb!=0 else ''
            
        cc = latexify(self.cc, self.precision)
        if self.cc>0 and (self.aa!=0 or self.bb!=0):
            stdForm += ' + '
        if self.cc!=0 or (self.aa==0 and self.bb==0):
            stdForm += cc
            
        return stdForm
    
    """@classmethod"""
    # def fromPts(cls, pt1, pt2, variable=x, label='f', domain=S.Reals, precision=2):
    #     """
    #     Construct linear function object from two ordered pairs.

    #     Parameters
    #     ----------
    #         pt1, pt2 : list or tuple
    #         variable : sympy symbol, default=x
    #         label : str, default='f'
    #     """

    #     pt1, pt2 = sympify([pt1, pt2])
        
    #     slope = (pt2[1] - pt1[1])/(pt2[0] - pt1[0])
        
    #     if pt1[0]==0:
    #         return cls(slope, pt1[1], variable, label, domain, precision)
    #     elif pt2[0]==0:
    #         return cls(slope, pt2[1], variable, label, domain, precision)
    #     else:
    #         return cls(slope, pt1, variable, label, domain, precision)

    def subs(self, input):
        """Return evaluation of function at input."""

        return self.expr.subs(self.variable, sympify(input))

    def subSet(self, preimage):
        """Return image of set."""

        return ImageSet(Lambda(self.variable, self.expr), preimage).intersect(S.Reals)

    def getRange(self):
        """Return range of function."""

        return self.subSet(self.domain)
        
    # def solve(self, output):
        # if self.slope == 0:
            # return self.domain if output==self.intercept else None
        # else:
            # return (sympify(output)-self.intercept)/self.slope

    def getStdForm(self, notation='func'):
        """Return LaTeX formatted standard form."""

        if notation == 'func':
            return f'{self.label}({self.variable}) = ' + str(self)
        else:
            return notation + '=' + str(self)

    # def getPtSlope(self, pt=1, notation='func'):
        # """
        # Return LaTeX formatted point-slope form.
        
        # Parameter
        # ---------
            # pt : number or list/tuple
                # Independent variable value or ordered pair.
        # """

        # pt = sympify(pt)
        # if type(pt)!=list:
            # pt = [pt] + [self.subs(pt)]
        
        # ptTex = latexify(pt, self.precision)
        
        # if self.slope in [0,1]:
            # ptSlope = ''
        # elif self.slope == -1:
            # ptSlope = '-'
        # else:
            # ptSlope = latexify(self.slope, self.precision)
            
        # if self.slope!=0:
            # ptSlope += r'\left(' + latex(self.variable)
            # ptSlope += '+' + ptTex[0][1:] if pt[0]<0 else '-' + ptTex[0]
            # ptSlope += r'\right)'
        
        # if pt[1]>0 and self.slope!=0:
            # ptSlope += ' + '
        # if pt[1]!=0 or self.slope==0:
            # ptSlope += ptTex[1]
        
        # if notation == 'func':
            # return fr'{self.label}({self.variable}) = ' + ptSlope
        # else:
            # return notation + '=' + ptSlope
        
    def getTable(self, nums, vals=True, vertical=True, labels=None):
        """
        Return LaTeX formatted table at given positions with or without blanks.
        
        Parameters
        ----------
            nums : list of ints
            vals : bool or list of bools or nonempty/empty strings ('')
        """
        
        vals = len(nums)*[vals] if type(vals)==bool else vals
        images = [signify(latexify(self.subs(jj), self.precision)) for jj in nums]
        images = [jj if vals[count] else r'\phantom{' + jj + r'}' for count,jj in enumerate(images)]
        
        nums = sympify(nums)
        nums = [signify(latexify(jj, self.precision)) for jj in nums]
        
        if labels == None:
            labels = [signify(str(self.variable)), fr'${self.label}({self.variable})$']
        if vertical:
            header = labels
            rows = list(zip(nums,images))
        else:
            header = [labels[0]] + list(nums)
            rows = [[labels[1]] + images]
        
        return tableGenerator(header, rows)

In [ ]:
class PWFunc():
    """
    Represent a piecewise function.

    Attributes
    ----------
        variable : sympy symbol, default=x
        label : str, default='f'
        expr : sympy expression
        domain : sympy set
    """

    def __init__(self, *pairs, variable=x, label='f'):
        """
        Construct piecewise function object from expr-interval pairs.

        Parameters
        ----------
            pairs : array of expr-interval pairs
            variable : sympy symbol, default=x
            label : str, default='f'
        """

        self.variable = variable
        self.label = label
        
        exprs, intervals = zip(*pairs)
        intervals = list(map(makeInterval, intervals))
        self.pairs = list(zip(exprs, intervals))
        
        pieces = [[row[0], Contains(variable, row[1])] for row in self.pairs]
        self.expr = Piecewise(*pieces)
        
        self.domain = Union(*intervals)
    
    def __str__(self):
        """Return LaTeX."""
        # TODO: relational instead of intervals

        return fr'{self.label}({self.variable}) = ' + latex(self.expr)

    def subs(self, input):
        """Return evaluation of function at input."""

        return nsimplify(self.expr.subs(self.variable, input))

    def subSet(self, preimage):
        """Return image of set."""
        
        image = S.EmptySet
        for expr, interval in self.pairs:
            image = Union(image, ImageSet(Lambda(self.variable,expr),interval.intersect(preimage)))

        return image.intersect(S.Reals)

    def getRange(self):
        """Return range of function."""

        image = S.EmptySet
        for expr, interval in self.pairs:
            image = Union(image, ImageSet(Lambda(self.variable,expr),interval))

        return image.intersect(S.Reals)
        
    def solve(self, output):
        """Return preimage of given output."""
        solns = []
        
        for expr, interval in self.pairs:
            answers = solve(expr-output, self.variable)
            for ans in answers:
                if ans in interval:
                    solns.append(ans)
        
        return list(set(solns))
    
    def getLatex(self):
        """Return LaTeX."""

        return str(self)

    def getFeats(self, feat='all', expr='latex'):           # TODO: INC/DEC
        features = dict()
        if feat in ['domain','all']:
            features['domain'] = self.domain
        if feat in ['range','all']:
            features['range'] = self.getRange()
        # if feat in ['change','all']:
        #     features['inc'] = 
        #     features['dec'] = 
        #     features['const'] = 
        if feat in ['extrema','all']:
            rangeset = func.getRange()
            features['mins'] = [func.solve(rangeset.inf), rangeset.inf]
            features['maxes'] = [func.solve(rangeset.sup), rangeset.sup]

        return latexify(features) if expr=='latex' else features

In [ ]:
class ArithSeq(LinFunc):
    """
    Represent an arithmetic sequence.

    Attributes
    ----------
        common : number (int, float, or Rational)
            Common difference.
        start : number
            First term (a_1 or f(1)).
        label : str, default='f'
        func : LinFunc
        terms : dict
    """

    def __init__(self, common, pt, variable=n, label='f', domain=S.Integers, precision=2):
        """
        Construct arithmetic sequence object from common difference and first term.

        Parameters
        ----------
            common : number (int, float, or Rational)
                Common difference.
            pt : number or list
                First term (a_1 or f(1)).
            label : str, default='f'
        """
        
        super().__init__(common, pt, variable, label, domain, precision)
        
        self.common = self.slope
        self.start = self.subs(1)
        self.terms = {0: self.intercept, 1: self.start, 2: self.subs(2), 3: self.subs(3)}
    
    def findTerm(self, num: int):
        """Return term at given index."""

        if num in self.terms.keys():
            return self.terms[num]
        else:
            return self.subs(num)
    
    def findTermNum(self, end):
        """Return index of given term."""

        if end in self.terms.values():
            return list(self.terms.keys())[list(self.terms.values()).index(end)]
        else:
            return self.solve(end)
    
    def getTerms(self, num: int, startnum=0):
        """Return dict of terms."""

        minNum = min(self.terms.keys())
        maxNum = max(self.terms.keys())
        if num > maxNum:
            for jj in range(maxNum+1, num+1):
                self.terms[jj] = self.terms[jj-1] + self.common
        if startnum < minNum:
            for jj in range(minNum-1, startnum-1, -1):
                self.terms[jj] = self.terms[jj+1] - self.common
        return self.terms
    
    def getExplicit(self, term=1):
        """
        Return LaTeX formatted explicit representation.
        
        Parameter
        ---------
            term : number or list/tuple
                Index of term or ordered pair.
        """

        return self.getSlopeInt() if term==0 else self.getPtSlope(term)
    
    def getRecursive(self, startnum=1):
        """
        Return LaTeX formatted recursive representation.

        Parameter
        ---------
            startnum : int
        """

        term = self.findTerm(startnum) if startnum!=1 else self.start
        recursive = fr'{self.label}(n) = {self.label}(n-1)'
        if self.common >= 0:
            recursive += ' + ' + latexify(self.common, self.precision)
        else:
            recursive += latexify(self.common, self.precision)
        recursive += fr',\quad {self.label}({startnum}) = {latexify(term, self.precision)}'
        return recursive
        
    def getSeqStr(self, nums=[1,2,3]):
        """
        Return LaTeX formatted list at given positions with or without blanks.
        
        Parameter
        ---------
            nums : list of ints or empty strings ('')
        """
        
        blank = r'\underline{\hspace{4mm}}'
        result = [latexify(self.findTerm(jj), self.precision) if type(jj)==int else blank for jj in nums]
        
        return (r',\ ').join(result) + r', \ldots'

class GeoSeq(ExpFunc):
    """
    Represent a geometric sequence.

    Attributes
    ----------
        common : number (int, float, or Rational)
            Common ratio.
        start : number
            First term (a_1 or f(1)).
        label : str, default='f'
        func : ExpFunc
        terms : dict
    """

    def __init__(self, common, pt, variable=n, label='f', domain=S.Integers, precision=2):
        """
        Construct geometric sequence object from common ratio and first term.

        Parameters
        ----------
            common : number (int, float, or Rational)
                Common ratio.
            start : number
                First term (a_1 or f(1)).
            label : str, default='f'
        """
        
        super().__init__(common, pt, variable, label, domain, precision)

        self.common = self.base
        self.start = self.subs(1)
        self.terms = {0: self.intercept, 1: self.start, 2: self.subs(2), 3: self.subs(3)}
    
    def findTerm(self, num: int):
        """Return term at given index."""

        if num in self.terms.keys():
            return self.terms[num]
        else:
            return self.subs(num)
    
    def findTermNum(self, end):
        """Return index of given term."""

        if end in self.terms.values():
            return list(self.terms.keys())[list(self.terms.values()).index(end)]
        else:
            return self.solve(end)
    
    def getTerms(self, num: int, startnum=0):
        """Return list of terms."""

        minNum = int(min(self.terms.keys()))
        maxNum = int(max(self.terms.keys()))
        if num > maxNum:
            for jj in range(maxNum+1, num+1):
                self.terms[jj] = self.terms[jj-1]*self.common
        if startnum < minNum:
            for jj in range(minNum-1, startnum-1, -1):
                self.terms[jj] = Rational(self.terms[jj+1],self.common)
        return self.terms
    
    def getExplicit(self, term=1):
        """
        Return LaTeX formatted explicit representation.
        
        Parameter
        ---------
            term : number or list/tuple
                Index of term or ordered pair.
        """

        return self.getBaseInt() if term==0 else self.getPtBase(term)
    
    def getRecursive(self, startnum=1):
        """
        Return LaTeX formatted recursive representation.

        Parameter
        ---------
            startnum : int
        """

        term = self.findTerm(startnum) if startnum!=1 else self.start
        recursive = fr'{self.label}(n) = {latexify(self.common, self.precision)}\cdot {self.label}(n-1)'
        recursive += fr',\quad {self.label}({startnum}) = {latexify(term, self.precision)}'
        return recursive
        
    def getSeqStr(self, nums=[1,2,3]):
        """
        Return LaTeX formatted list at given positions with or without blanks.
        
        Parameter
        ---------
            nums : list of ints or empty strings ('')
        """
        
        blank = r'\underline{\hspace{4mm}}'
        result = [latexify(self.findTerm(jj), self.precision) if type(jj)==int else blank for jj in nums]
        
        return (r',\ ').join(result) + r', \ldots'


###Generators and printers

In [ ]:
def chooseFloat(package):
  floatA = random.choice(package)
  return floatA

def getFloat(min=-10,max=10,roundby=2):
  return round(random.uniform(min, max), roundby)

def getInt(min= -4, max = 4, exclude=[0]):
  intA = random.choice([x for x in range(min,max+1) if x not in exclude])
  return sympify(intA)

def getVar(exclude=None):
  return random.choice([vrbl for vrbl in variables if vrbl not in exclude])

def getFrac(denommin= -4, denommax= 4, numermin = 1, numbermax = 4 , exclude = 0):# exclude 1 in denom later
  top = random.choice([x for x in range(numermin,numbermax+1) if x != exclude])
  bot = random.choice([x for x in range(denommin,denommax+1) if x != exclude])
  return latex(simplify(Rational(top,bot),evaluate=False))

"""###Table"""

def tableGenerator(header, rows, cols_align=None, cols_valign=None, expr='latex', table_env=False, stretch=1.5):
    """
    Generate a table.

    Parameters
    ----------
        header : list or tuple
        rows : 2-dimensional array
        cols_align : list or tuple, default=None
            List of column alignments ('l','c', or 'r'); if ``None``, all set to 'c'.
        expr : str, default='latex'
            LaTeX formatted.
        table_env : boolean, default=True
            Enclose in floating table environment (\begin{table}...).

    Returns
    -------
        str
    """

    table = Texttable()
    if cols_align == None:
        cols_align = len(header)*['c']
    if cols_valign == None:
        cols_valign = len(header)*['m']
    table.set_cols_align(cols_align)
    table.set_cols_valign(cols_valign)
    table.add_rows([header] + rows)

    if expr == 'latex':
        table = latextable.draw_latex(table)
        if not table_env:
            table = table.replace('\\begin{table}\n','').replace('\n\\end{table}','')
        if stretch == 1:
            return table
        else:
            return r'{\renewcommand{\arraystretch}' + brackify(stretch) + table + r'}'
    else:
        return table.draw()
        
def makeInterval(string):
    if type(string)==Interval:
        return string
    left_open = (string[0]=='(')
    right_open = (string[-1]==')')
    start, end = sympify(string[1:-1].split(','))
    return Interval(start, end, left_open, right_open)

In [ ]:
def signify(data):
    if type(data)==list:
        return [signify(jj) for jj in data]
    else:
        return '$' + str(data) + '$'
    
def latexify(data, precision=2):
    if type(data)==list:
        return [latexify(num, precision) for num in data]
    elif type(data)==dict:
        return {k:latexify(v, precision) for k,v in data.items()}
    elif type(data) in [Integer,int]:
        return f'{int(data):,}'
    elif type(data) in [Float,float]:
        return f'{data:,.{precision}f}'
    elif type(data)==Rational and abs(data.q)>999:
        return f'{Float(data):,.{precision}f}'
    elif type(data)==Mul:
        a,b,c = Wild('a'),Wild('b'),Wild('c')
        data = data.match(a*b**c)
        data = data[a]*UnevaluatedExpr(real_root(data[b],data[c].q))**data[c].p
        return latex(data)
    elif type(data)==str:
        return data
    else:
        return latex(data)

def brackify(data):
    return r'{' + str(data) + r'}'
        
def minipagify(left, right, vspace='0.25in'):
    string = '\\begin{minipage}[t]{0.5\\textwidth}\n'
    string += left + '\n\\end{minipage}\n'
    string += '\\begin{minipage}[t]{0.5\\textwidth}\n'
    string += right + '\n\\end{minipage}\n\n'
    string += f'\\vspace{brackify(vspace)}\n'
    return string

###Graphers

In [ ]:
def startGraph(xmin=-10,xmax=10,ymin=-10,ymax=10):
    xstart = 5*(xmin//5)
    ystart = 5*(ymin//5)
    xtick = brackify(f'{xstart},{xstart+5},...,{xmax+5}')
    ytick = brackify(f'{ystart},{ystart+5},...,{ymax+5}')

    tex = r'\begin{tikzpicture}\begin{axis}[mmt axis style,'
    tex += fr'xmin={xmin},xmax={xmax},xtick={xtick},ymin={ymin},ymax={ymax},ytick={ytick},]'
    return tex

def endGraph():
    return r'\end{axis}\end{tikzpicture}'

def emptyGraph(xmin=-10,xmax=10,ymin=-10,ymax=10):
    return startGraph(xmin,xmax,ymin,ymax) + endGraph()

def drawPt(pt, draw='black', fill='black', shape='circle', size='2pt'):
    return fr'\fill[draw={draw},fill={fill}] (axis cs: {pt[0]},{pt[1]}) {shape}({size});'

def drawScatter(pts, draw='black', fill='black', shape='', size='2pt'):
    """
    Return LaTeX to draw list of points on a graph.

    Parameters
    ----------
        pts : list
            List of coordinate pairs.
        draw : string, default='black'
            Color of mark boundaries.
        fill : string, default='black'
            Color of mark fills.
        shape : string, default=''
            Shape of marks, default='' gives circle.
                Other options include 'triangle','square','diamond,'star','otimes'.
        size : string, default='2pt'
    """

    pts = ' '.join([str(tuple(jj)) for jj in pts])
    return fr'\addplot[only marks,draw={draw},fill={fill},mark={shape}*,mark size={size}] coordinates {brackify(pts)};'
    
def drawLinear(expr,xmin,xmax,color='black',style='solid'):
    string = fr'\addplot[{style},domain={xmin}:{xmax}, color={color},]{brackify(expr)};'
    return string

def drawCurve(expr, inMin, inMax, LHS='y', color='black', style='solid', width='thick', label=None, fillName=None):
    """
    Return LaTeX to draw a curve on a graph.

    Parameters
    ----------
        expr : string or sympy expression
            Right-hand side of equation
        inMin : number
            Minimum value of independent variable.
        inMax : number
            Maximum value of independent variable.
        LHS : string, default='y'
            Left-hand side of equation.
        color : string, default='black'
        style : string, default='solid'
        width : string, default='thick'
        label : list or string, default=None
            If list, format as [<label>, <rel. pos.>, <angle>] where rel. pos.
                is relative position as a float in [0,1] and angle is given in degrees.
            If string, default rel. pos.=0.7 and default angle=45.
        fillName : string, default=None
            Name of curve used for fillbetween library (needed only for inequality graphers).
    """
    
    string = fr'\addplot[{color},{style},{width},domain={inMin}:{inMax},samples=100'
    string += r'] ' if fillName==None else fr',name path={fillName}] '

    variables = sympify(expr).free_symbols
    expr = str(expr).replace('**','^')

    if (variables=={x}) or (LHS in ['y',y]):        # Function of x
        string += brackify(expr)
    elif (variables=={y}) or (LHS in ['x',x]):      # Function of y
        expr = expr.replace('y','x')
        string += f'({brackify(expr)},{brackify(x)})'

    if label==None:
        return string + r';'
    else:
        if type(label) != list:
            label = [label, 0.7, 45]
        return string + fr' node[pos={label[1]},pin={label[2]}:{signify(label[0])}]' + r' {};'

def drawSlopeTri(pt1, pt2):
    string = fr'\draw[dashed] (axis cs: {pt1[0]},{pt1[1]}) -- (axis cs: {pt1[0]},{pt2[1]});'
    string += fr'\draw[dashed] (axis cs: {pt1[0]},{pt2[1]}) -- (axis cs: {pt2[0]},{pt2[1]});'
    return string

def shadeRegion(xmin, xmax, lower, upper, color='blue!30', shading='opacity=0.5'):
    domain = fr'domain={xmin}:{xmax}'
    string = fr'\addplot[{color},{shading}] fill between'
    string += fr'[of={lower} and {upper},soft clip={brackify(domain)}];'
    return string

###Word problems

In [ ]:
import names, pandas
from num2words import num2words

DIFF, RATIO, SEQ_1, SEQ_2, SEQ_3 = symbols('DIFF RATIO SEQ_1 SEQ_2 SEQ_3')
INTA, INTB, FLOATA, FLOATB = symbols('INTA INTB FLOATA FLOATB')

Lin_probs = pandas.read_csv('Lin_probs.csv')
Lin_probs = Lin_probs[['Question','Answer1','Answer2','Controller','In','Out']]

Exp_probs = pandas.read_csv('Exp_probs.csv')
Exp_probs = Exp_probs[['Question','Answer1','Answer2','Controller','In','Out']]

def getControls(controls):
    params = dict()

    for word in controls:
        word, range = word.split('=')
        range = range.replace('[','').replace(']','').split(',')
        mnm, mxm = sympify(range[:2])
        params[word] = getFloat(mnm,mxm) if type(mnm)==Float else getInt(mnm,mxm)

    return params

def getParam(word, params, kind):
    if word == 'LASTNAME':
        return names.get_last_name()
    elif word == 'FIRSTNAME':
        return names.get_first_name()
    elif word == 'FIRSTNAME_MALE':
        return names.get_first_name(gender='male')
    elif word == 'FIRSTNAME_FEMALE':
        return names.get_first_name(gender='female')
    elif word == 'SEQ_2':
        if kind == 'lin':
            return params['SEQ_1'] + params['DIFF']
        elif kind == 'exp':
            return params['SEQ_1']*params['RATIO']
    elif word == 'SEQ_3':
        if kind == 'lin':
            return 2*params['SEQ_2'] - params['SEQ_1']
        elif kind == 'exp':
            return params['SEQ_2']**2/params['SEQ_1']
    elif word == 'MULT_PHRASE':
        if 'RATIO' not in params.keys():
            params['RATIO'] = randint(2,5)
        phrase = 'twice' if params['RATIO']==2 else num2words(params['RATIO']) + ' times'
        return phrase
    elif word == 'GROWTH_PHRASE':
        growths = {2: 'doubles', 3: 'triples', 4: 'quadruples'}
        if 'RATIO' not in params.keys():
            params['RATIO'] = randint(2,5)
        if params['RATIO'] in growths.keys():
            phrase = growths[params['RATIO']]
        else:
            phrase = 'grows by a factor of ' + num2words(params['RATIO'])
        return phrase

def getWordValue(word, params, kind, string=True):
    symbol1 = ''
    symbol2 = ''
    if (not word[0].isalnum()) and word[0]!='(':
        symbol1 = '\\' + '$' if word[0] in ['$','£'] else word[0]
        word = word[1:]
    if (not word[-1].isalnum()) and word[-1]!=')':
        symbol2 = '\\' + word[-1] if word[-1]=='%' else word[-1]
        word = word[:-1]

    if '(' in word:
        word = sympify(word).subs(params.items())
    elif word.isnumeric():
        word = sympify(word)
    else:
        if word not in params.keys():
            params[word] = getParam(word, params, kind)
        word = params[word]
    
    return symbol1 + str(latexify(word)) + symbol2 if string else word

def makeWordProb(kind='lin', expr='latex'):
    if kind == 'lin':
        data = Lin_probs.loc[randint(0,len(Lin_probs)-1)]
    elif kind == 'exp':
        data = Exp_probs.loc[randint(0,len(Exp_probs)-1)]
    else:
        data = Neither_probs.loc[randint(0,len(Neither_probs)-1)]

    problem = data[0].split()
    solns = data[1:3].copy()
    controls = data[3].split()
    labels = list(data[4:6].values)

    params = getControls(controls)

    expressions = []
    for count,word in enumerate(problem):
        if word.isupper() and len(word)>2:
            if '(' in word:
                expressions.append([count,word])
            else:
                problem[count] = getWordValue(word, params, kind)
    for count,word in expressions:
        problem[count] = getWordValue(word, params, kind)
    
    for jj in [0,1]:
        solns[jj] = solns[jj].split(',')
        if len(solns[jj]) > 1:
            solns[jj] = [ getWordValue(solns[jj][0], params, kind, string=False), getWordValue(solns[jj][1], params, kind, string=False) ]
        else:
            solns[jj] = getWordValue(solns[jj][0], params, kind, string=False)

    return ' '.join(problem), solns, labels

FileNotFoundError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

##1.4.1 Arithmetic - Explicit & Recursive

### section 1
#### instruction : Find the next three terms in each sequence. Identify the common difference. Write a recursive function and an explicit function for each sequence.

1) easy start(-10~10) 
- ratio : ( -5~5 except 0)
- a. 3,7,11,15,19,___,___,___,...
- b. Common Difference:
- c. Recursive Function: 
- d. Explicit Function:

2) medium start(-20~20) 
- ratio : ( -10 ~ -5 and 5 ~ 10 )
- a. 3,7,11,15,19,___,___,___,...
- b. Common Difference:
- c. Recursive Function: 
- d. Explicit Function:

3) hard start(-20~20) 
- ratio : ( -30 ~ -10 and 10 ~ 30 )
- a. 3,7,11,15,19,___,___,___,...
- b. Common Difference:
- c. Recursive Function: 
- d. Explicit Function:

In [ ]:
import sympy
import random

def arith_problem_terms_gen(start, ratio):
    function = []
    function.append(str(start))
    for i in range(random.randint(3,7)):
        function.append(str(int(function[-1]) + int(ratio)))
    return function


def explicit_function_formatter(ratio, variable, start, notation):
    return str(sympy.latex(sympy.sympify(fr'{notation} * ({variable} - 1) + {ratio}', evaluate = False), fold_short_frac=False))  + fr", {notation}(1) = {start}"


def recursive_function_formatter(start, variable, ratio, notation):
    return sympy.latex(sympy.sympify(fr'{start} + ({variable} - 1) * {ratio}', evaluate = False), fold_short_frac=False)


def dollar_signify(input):
    return("$" + input + "$")


def Arithmetic_Explicit_Recursive(difficulty=1, expr="latex"):
    """variables.append(random.choice(choices))
            sym_1 = random.choice(addition_expression)
            numerals.append(str(random.randint(2, 10)))
            numerals.append(str(random.choice(negative)) +  str(random.randint(1, 10)))

            x = sp.symbols(str(variables[0]))
            if random.randint(1,2) == 1:
                output = '{} {} {}'.format(str(variables[0]), sym_1, str(numerals[0]))
            else:
                output = '{} {} {}'.format(str(numerals[0]), sym_1, str(variables[0]))
            other = '{}'.format(str(numerals[1]))
            equation  = sp.Eq(sp.sympify(output), sp.sympify(other))

            problemsets.append(sp.latex(equation))
            answerset.append(sp.solve(equation, x))"""
    variable = random.choice(['x', 'y', 'a', 'b', 'z', 'p', 't'])
    notation = random.choice(['E', 'K', 'U', 'R', 'D', 'W', 'G', 'H', 'V'])
    function_start = notation + "(" + variable + ")"
    problem = ""
    answer = ""
    if difficulty == 1:     # Easy
        start = random.randint(-10, 10)
        if random.randint(1,2) == 1:
            ratio = random.randint(-5, -1)
        else:
            ratio = random.randint(1, 5)

        problem = arith_problem_terms_gen(str(start), str(ratio))
        problem = ', '.join(problem)
        problem = str(problem) + ", ..."

        answer = "Common Difference: " + str(ratio), "Explicit Function: " + dollar_signify(function_start + " = " + recursive_function_formatter(start, variable, ratio, notation)), "Recursive Function: " + dollar_signify(function_start + " = "+ explicit_function_formatter(start, variable, ratio, notation))
        """
        ratio : ( -5~5 except 0)
        a. 3,7,11,15,19,,,___,... // generate this (prev figure)
        b. Common Difference:
        c. Recursive Function:
        d. Explicit Function
        """
    elif difficulty == 2:   # Medium
        """2) medium start(-20~20)
        ratio : ( -10 ~ -5 and 5 ~ 10 )
        a. 3,7,11,15,19,,,___,...
        b. Common Difference:
        c. Recursive Function:
        d. Explicit Function:"""
        start = random.randint(-20, 20)
        if random.randint(1,2) == 1:
            ratio = random.randint(-10, -5)
        else:
            ratio = random.randint(5, 10)
        problem = arith_problem_terms_gen(str(start), str(ratio))
        problem = ', '.join(problem)
        problem = str(problem) + ", ..."

        answer = "Common Difference: " + str(ratio), "Explicit Function: " + dollar_signify(function_start + " = " + recursive_function_formatter(start, variable, ratio, notation)), "Recursive Function: " + dollar_signify(function_start + " = "+ explicit_function_formatter(start, variable, ratio, notation))


    elif difficulty == 3:   # Hard
        """3) hard start(-20~20)
        ratio : ( -30 ~ -10 and 10 ~ 30 )
        a. 3,7,11,15,19,,,___,...
        b. Common Difference:
        c. Recursive Function:
        d. Explicit Function:"""
        start = random.randint(-20, 20)
        if random.randint(1,2) == 1:
            ratio = random.randint(-30, -10)
        else:
            ratio = random.randint(10, 30)
        problem = arith_problem_terms_gen(str(start), str(ratio))
        problem = ', '.join(problem)
        problem = str(problem) + ", ..."

        answer = "Common Difference: " + str(ratio), "Explicit Function: " + dollar_signify(function_start + " = " + recursive_function_formatter(start, variable, ratio, notation)), "Recursive Function: " + dollar_signify(function_start + " = "+ explicit_function_formatter(start, variable, ratio, notation))
    
    return problem, r'\\'.join(answer)
for i in range(20):
    a,b, = Arithmetic_Explicit_Recursive(1, "latex")

    print(b)

Common Difference: -5\\Explicit Function: $U(z) = \left(z - 1\right) \left(-5\right) + 0$\\Recursive Function: $U(z) = U \left(z - 1\right) + 0, U(1) = -5$
Common Difference: -4\\Explicit Function: $K(p) = \left(p - 1\right) \left(-4\right) - 3$\\Recursive Function: $K(p) = K \left(p - 1\right) - 3, K(1) = -4$
Common Difference: -1\\Explicit Function: $V(a) = 7 + \left(a - 1\right) \left(-1\right)$\\Recursive Function: $V(a) = V \left(a - 1\right) + 7, V(1) = -1$
Common Difference: -3\\Explicit Function: $D(a) = \left(a - 1\right) \left(-3\right) - 7$\\Recursive Function: $D(a) = D \left(a - 1\right) - 7, D(1) = -3$
Common Difference: 1\\Explicit Function: $K(t) = \left(t - 1\right) 1 + 1$\\Recursive Function: $K(t) = K \left(t - 1\right) + 1, K(1) = 1$
Common Difference: -5\\Explicit Function: $U(b) = \left(b - 1\right) \left(-5\right) - 10$\\Recursive Function: $U(b) = U \left(b - 1\right) - 10, U(1) = -5$
Common Difference: 3\\Explicit Function: $U(t) = \left(t - 1\right) 3 - 9$\\Re

In [ ]:
def getSeqAnswer(seq, prob='common', blanks=False, expr="latex", numTerms=6):
    if expr=="latex":
        answer = ''
        if prob in ['common','all'] :
            answer += '$d = ' if type(seq)==ArithSeq else '$r = '
            answer += latexify(seq.common, seq.precision) + '$'
        if prob in ['explicit','all']:
            if len(answer)>0:
                answer += r' \newline '
            answer += '$' + seq.getExplicit() + r' \quad\text{or}\quad ' + seq.getExplicit(0) + '$'
        if prob in ['recursive','all']:
            if len(answer)>0:
                answer += r' \newline '
            answer += '$' + seq.getRecursive() + r' \quad\text{or}\quad ' + seq.getRecursive(0) + '$'
        
        if blanks:
            if len(answer)>0:
                answer += r' \newline '
            answer += signify(seq.getSeqStr(range(1,numTerms+1))) 
    else:
        seq.getTerms(numTerms)
        answer = seq

    return answer

In [ ]:
def Arithmetic_Sequence_Problem(difficulty=1, prob='common', blanks=False, expr="latex"):
    if difficulty == 1: # easy
        if randint(0,1):
            arith = ArithSeq(randint(1,10), randint(1,10))
        else:
            arith = ArithSeq(randint(-10,-1), randint(-10,-1))
    elif difficulty == 2:
        if randint(0,1):
            arith = ArithSeq(randint(1,20), randint(1,20))
        else:
            arith = ArithSeq(randint(-20,-1), randint(-20,-1))   
  
    else:
        if randint(0,1):
            arith = ArithSeq(randint(1,20), randint(1,30))
        else:
            arith = ArithSeq(randint(-20,-1), randint(-30,-1))

    nums = [1,2,'','','',''] if blanks else [1,2,3,4,5]
    problem = signify(arith.getSeqStr(nums))
    answer = getSeqAnswer(arith, prob, blanks, expr)
            
    return problem, answer

for i in range(5):
    problem, answer = Arithmetic_Sequence_Problem(1, 'all', blanks=True)
    print(problem, r'\\')
    print(answer, r'\\')

for i in range(5):
    problem, answer = Arithmetic_Sequence_Problem(2, 'all', blanks=True)
    print(problem, r'\\')
    print(answer, r'\\')

for i in range(5):
    problem, answer = Arithmetic_Sequence_Problem(3, 'all', blanks=True)
    print(problem, r'\\')
    print(answer, r'\\')

$20,\ 30,\ \underline{\hspace{4mm}},\ \underline{\hspace{4mm}},\ \underline{\hspace{4mm}},\ \underline{\hspace{4mm}}, \ldots$ \\
$d = 10$ \newline $f(n) = 10\left(n-1\right) + 20 \quad\text{or}\quad f(n) = 10n + 10$ \newline $f(n) = f(n-1) + 10,\quad f(1) = 20 \quad\text{or}\quad f(n) = f(n-1) + 10,\quad f(0) = 10$ \newline $20,\ 30,\ 40,\ 50,\ 60,\ 70, \ldots$ \\
$19,\ 28,\ \underline{\hspace{4mm}},\ \underline{\hspace{4mm}},\ \underline{\hspace{4mm}},\ \underline{\hspace{4mm}}, \ldots$ \\
$d = 9$ \newline $f(n) = 9\left(n-1\right) + 19 \quad\text{or}\quad f(n) = 9n + 10$ \newline $f(n) = f(n-1) + 9,\quad f(1) = 19 \quad\text{or}\quad f(n) = f(n-1) + 9,\quad f(0) = 10$ \newline $19,\ 28,\ 37,\ 46,\ 55,\ 64, \ldots$ \\
$17,\ 24,\ \underline{\hspace{4mm}},\ \underline{\hspace{4mm}},\ \underline{\hspace{4mm}},\ \underline{\hspace{4mm}}, \ldots$ \\
$d = 7$ \newline $f(n) = 7\left(n-1\right) + 17 \quad\text{or}\quad f(n) = 7n + 10$ \newline $f(n) = f(n-1) + 7,\quad f(1) = 17 \quad\text{or}

##1.4.2 Find the slope. 

---
- startA = [-10,10]
- startB = [-20,20]
- floatA = [ 0.3 , 0.5 , 0.9]
- floatB = [ -9.9,9.9]
- intA = [-4,-3,-2,-1,1,2,3,4]
- intB = [-4,-3,-2,-1,1,2,3,4]
- intC = [-4,-3,-2,-1,1,2,3,4]
- intD = [-4,-3,-2,-1,1,2,3,4]
- intE = [-10,-9,-8,-7,-6,-5,-4,-3,-2,-1,1,2,3,4,5,6,7,8,9,10]
- intF = [-10,-9,-8,-7,-6,-5,-4,-3,-2,-1,1,2,3,4,5,6,7,8,9,10]
---

### Section 1
####instruction: Find the slope for each line based on the given representation.

- 1 . graph
  - case 1
  - problem : "linear equation display"
  - answer :  m = $\frac{2}{3}$
  - case 2
  - problem : "linear equation display"
  - answer :   m = $-\frac{2}{3}$
  - case 3
  - problem : "linear equation display"
  - answer :  m = $0$
  - case 4
  - problem : "linear equation display"
  - answer :   m = $\text{undefined}$
- 2 . table
  - case 1
  - problem : table
  - answer :  m = $2$
  - case 2
  - problem : table
  - answer :  m = $-2$
  - case 3
  - problem : table
  - answer :  m = $\text{undefined}$
  - case 4
  - problem : table
  - answer :  m = $ 0 $
- 3 . linear eq
  - case 1
  - problem : table
  - answer :  m = $2$
  - case 2
  - problem : table
  - answer :  m = $-2$
  - case 3
  - problem : table
  - answer :  m = $\text{undefined}$
  - case 4
  - problem : table
  - answer :  m = $ 0 $
- 4. 2 points given
  - case 1
  - positive slope
  - problem : $(2,2)$ and $(4,4)$
  - answer :  m = $1$
  - case 2
  - negative slope
  - problem : $(4,4)$ and $(-1,6)$
  - answer :  m = $\frac{5}{2}$
  - case 3
  - undefined
  - problem : $(3,2)$ and $(3,4)$
  - answer :  m = $\text{undefined}$
  - case 4
  - 0
  - problem : $(4,2) and (3,2)$
  - answer :  m = $0$

In [ ]:
# # Latex setup required in preamble

# \newcommand{\drawaline}[4]{
# \draw [extended line=1cm,stealth-stealth] (#1,#2)--(#3,#4);
# \fill [black](#1,#2) circle(4pt);
# \fill [black](#3,#4) circle(4pt);
# }

In [ ]:
"""
intA: [-5,5]

intB: [-5,5]

intC: [-5,5]

intD: [-5,5]

intA: [-5,5]

intB: [-5,5]

intC: [-5,5]

intD: [-5,5]

slopeA: [-3,3]

yintA: [-10,10]
"""
import random
import sympy
import fractions


def fix_zero_neg(input):
    if int(input) == 0:
        return 0 
    else:
        return input

# SAME AS THE ONE IN MODULE 9
def AGS_Find_The_Slope_Section_1(difficulty=1, expr="latex"):
    startgraph = r'\begin{tikzpicture}[scale=.5,extended line/.style={shorten >=-#1,shorten <=-#1},]\draw [help lines] (-6,-6) grid (6,6);'
    startgraph += r'\draw [<->](0,-6.2)--(0,6.2) node[right]{$y$};\draw [<->](-6.2,0)--(6.2,0) node[right]{$x$};'
    startgraph += r'\foreach \x in {-6,-5,-4,-3,-2,-1,1,2,3,4,5,6}\draw (\x,1pt) -- (\x,-3pt) node[anchor=north] {\x};'
    startgraph += r'\foreach \y in {-6,-5,-4,-3,-2,-1,1,2,3,4,5,6}\draw (1pt,\y) -- (-3pt,\y) node[anchor=east] {\y};'

    endgraph = r'\end{tikzpicture}'

    rise = ""
    run = ""
    problem = ""
    answer = ""
    point_1 = ""
    point_2 = ""
    point_3 = ""
    point_4 = ""
    negative = ["", "-"]

    intA = fix_zero_neg(str(random.choice(negative)) + str(random.randint(0, 5)))
    intB = fix_zero_neg(str(random.choice(negative)) + str(random.randint(0, 5)))
    intC = fix_zero_neg(str(random.choice(negative)) + str(random.randint(0, 5)))
    intD = fix_zero_neg(str(random.choice(negative)) + str(random.randint(0, 5)))
    intA_2 = fix_zero_neg(str(random.choice(negative)) + str(random.randint(0, 5)))
    intB_2 = fix_zero_neg(str(random.choice(negative)) + str(random.randint(0, 5)))
    intC_2 = fix_zero_neg(str(random.choice(negative)) + str(random.randint(0, 5)))
    intD_2 = fix_zero_neg(str(random.choice(negative)) + str(random.randint(0, 5)))

    if difficulty == 1:     # Easy
        if random.randint(1, 2) == 1:
            #point_1 = ( intA , 0 )
            #point_2 = ( intB , intC)
            while abs(int(intB) - int(intA)) == 0:
                intB = str(random.choice(negative)) + str(random.randint(0, 5))
                intA = str(random.choice(negative)) + str(random.randint(0, 5))
            problem = fr"\drawaline`{intA}~`{0}~`{intB}~`{intC}~"
            if abs(int(intB) - int(intA)) == 0:
                AGS_Find_The_Slope_Section_1(difficulty, expr)
            rise = int(intC)
            run = (int(intB) - int(intA))

        else:
            # point 3 ( 0 , intA_2)
            # point 4 ( intB_2 , intC_2)
            if abs(int(intB_2)) == 0:
                AGS_Find_The_Slope_Section_1(difficulty, expr)
            while abs(int(intB_2)) == 0:
                intB_2 = str(random.choice(negative)) + str(random.randint(0, 5))
            problem = fr"\drawaline`{0}~`{intA_2}~`{intB_2}~`{intC_2}~"
            rise = (int(intC_2) - int(intA_2))
            run = (int(intB_2))
        answer = fractions.Fraction(rise, run)
        problem = problem.replace('`', "{")
        problem = problem.replace('~', "}")
        if answer == 0:
            answer = "Zero Slope" # Horizontal
        elif abs(run) == 0:
            answer = "Undefined Slope" # Vertical

    elif difficulty == 2 or difficulty == 3:    # Medium or hard
        if random.randint(1, 2) == 1:
            #point_1 = ( intA , 0 )
            #point_2 = ( intB , intC)
            while abs(int(intB) - int(intA)) == 0:
                intB = str(random.choice(negative)) + str(random.randint(0, 5))
                intA = str(random.choice(negative)) + str(random.randint(0, 5))
            problem = fr"\drawaline`{intA}~`{intD}~`{intB}~`{intC}~"
            if abs(int(intB) - int(intA)) == 0:
                AGS_Find_The_Slope_Section_1(difficulty, expr)
            rise = int(intC) - int(intD)
            run = (int(intB) - int(intA))

        else:
            # point 3 ( 0 , intA_2)
            # point 4 ( intB_2 , intC_2)
            if abs(int(intB_2)) == 0:
                AGS_Find_The_Slope_Section_1(difficulty, expr)
            while abs(int(intB_2) - int(intD_2)) == 0:
                intB_2 = str(random.choice(negative)) + str(random.randint(0, 5))
                intD_2 = str(random.choice(negative)) + str(random.randint(0, 5))
            problem = fr"\drawaline`{intD_2}~`{intA_2}~`{intB_2}~`{intC_2}~"
            rise = (int(intC_2) - int(intA_2))
            run = (int(intB_2) - int(intD_2))

        answer = fractions.Fraction(rise, run)
        problem = problem.replace('`', "{")
        problem = problem.replace('~', "}")
        if answer == 0:
            answer = "Zero Slope" # Horizontal
        elif abs(run) == 0:
            answer = "Undefined Slope" # Vertical

    problem = startgraph + problem + endgraph

    return(problem, answer)


for i in range(3):
    problem, answer = AGS_Find_The_Slope_Section_1(1, "latex")
    print(problem, answer)

for i in range(3):
    problem, answer = AGS_Find_The_Slope_Section_1(2, "latex")
    print(problem, answer)

\begin{tikzpicture}[scale=.5,extended line/.style={shorten >=-#1,shorten <=-#1},]\draw [help lines] (-6,-6) grid (6,6);\draw [<->](0,-6.2)--(0,6.2) node[right]{$y$};\draw [<->](-6.2,0)--(6.2,0) node[right]{$x$};\foreach \x in {-6,-5,-4,-3,-2,-1,1,2,3,4,5,6}\draw (\x,1pt) -- (\x,-3pt) node[anchor=north] {\x};\foreach \y in {-6,-5,-4,-3,-2,-1,1,2,3,4,5,6}\draw (1pt,\y) -- (-3pt,\y) node[anchor=east] {\y};\drawaline{-2}{0}{4}{0}\end{tikzpicture} Zero Slope
\begin{tikzpicture}[scale=.5,extended line/.style={shorten >=-#1,shorten <=-#1},]\draw [help lines] (-6,-6) grid (6,6);\draw [<->](0,-6.2)--(0,6.2) node[right]{$y$};\draw [<->](-6.2,0)--(6.2,0) node[right]{$x$};\foreach \x in {-6,-5,-4,-3,-2,-1,1,2,3,4,5,6}\draw (\x,1pt) -- (\x,-3pt) node[anchor=north] {\x};\foreach \y in {-6,-5,-4,-3,-2,-1,1,2,3,4,5,6}\draw (1pt,\y) -- (-3pt,\y) node[anchor=east] {\y};\drawaline{-3}{0}{4}{2}\end{tikzpicture} 2/7
\begin{tikzpicture}[scale=.5,extended line/.style={shorten >=-#1,shorten <=-#1},]\draw [hel

In [ ]:
from tabulate import tabulate
from texttable import Texttable

import latextable
import random
import fractions
from sympy import Rational

def problem_terms_gen(start, ratio, interateby):
    indexes = []
    terms = []
    terms.append(str(start))
    indexes.append(str(random.randint(-10, 10)))
    for i in range(random.randint(3,7)):
        terms.append(str(int(terms[-1]) + int(ratio)))
        indexes.append(str(int(indexes[-1]) + int(interateby)))
    return indexes, terms

def table_selector(rows):
    table = Texttable()
    table.set_cols_align(["c"] * 2)
    table.set_deco(Texttable.HEADER | Texttable.VLINES)
    table.add_rows(rows)
    if random.randint(1,2) == 1:
        #print('\nTabulate Latex:')
        return(tabulate(rows, headers='firstrow', tablefmt='latex'))
    else:
    #print('\nTexttable Latex:')
        return(latextable.draw_latex(table))

def AGS_Find_The_Slope_Section_2(difficulty=1, expr="latex"):
    problem = ""
    answer = ""
    x_layer = ""
    y_layer = ""

    if difficulty == 1:     # Easy
        start = random.randint(-10, 10)
        interateby = 1
        if random.randint(1,2) == 1:
            ratio = random.randint(-5, -1)
        else:
            ratio = random.randint(1, 5)
        x_layer, y_layer = problem_terms_gen(start, ratio, interateby)

        rows = []
        for i in range(len(x_layer)):
            rows.append([])
            rows[i].append(x_layer[i])
            rows[i].append(y_layer[i])
        rows.insert(0, ['x', 'y'])
        
        problem = table_selector(rows)
        answer = ratio

    elif difficulty == 2:   # Medium
        interateby = random.randint(-3, 3)
        start = random.randint(-15, 15)
        ratio = random.randint(-15, 15)
        x_layer, y_layer = problem_terms_gen(start, ratio, interateby)

        rows = []
        for i in range(len(x_layer)):
            rows.append([])
            rows[i].append(x_layer[i])
            rows[i].append(y_layer[i])
        rows.insert(0, ['x', 'y'])
        
        problem = table_selector(rows)
        if interateby != 0:
            answer = Rational(ratio , interateby)
        else: 
            answer = 0

    elif difficulty == 3:   # Hard
        start = random.randint(-50, 50)
        ratio = random.randint(-50, 50)
        interateby = random.randint(-5, 5)
        x_layer, y_layer = problem_terms_gen(start, ratio, interateby)

        rows = []
        for i in range(len(x_layer)):
            rows.append([])
            rows[i].append(x_layer[i])
            rows[i].append(y_layer[i])
        rows.insert(0, ['x', 'y'])
        
        problem = table_selector(rows)
        if interateby != 0:
            answer = Rational(ratio , interateby)
        else: 
            answer = 0

    if len(str(answer)) >= 8:
        AGS_Find_The_Slope_Section_2(difficulty, expr)
    return problem, answer

for i in range(3):
    problem, answer = AGS_Find_The_Slope_Section_2(1, "latex")
    print(problem, answer)

for i in range(3):
    problem, answer = AGS_Find_The_Slope_Section_2(2, "latex")
    print(problem, answer)

for i in range(3):
    problem, answer = AGS_Find_The_Slope_Section_2(3, "latex")
    print(problem, answer)

\begin{table}
	\begin{center}
		\begin{tabular}{c|c}
			x & y \\
			\hline
			0 & 3 \\
			1 & 7 \\
			2 & 11 \\
			3 & 15 \\
			4 & 19 \\
			5 & 23 \\
			6 & 27 \\
		\end{tabular}
	\end{center}
\end{table} 4
\begin{tabular}{rr}
\hline
   x &   y \\
\hline
  -3 &  -1 \\
  -2 &  -5 \\
  -1 &  -9 \\
   0 & -13 \\
   1 & -17 \\
   2 & -21 \\
\hline
\end{tabular} -4
\begin{table}
	\begin{center}
		\begin{tabular}{c|c}
			x & y \\
			\hline
			-9 & 1 \\
			-8 & -3 \\
			-7 & -7 \\
			-6 & -11 \\
			-5 & -15 \\
			-4 & -19 \\
			-3 & -23 \\
		\end{tabular}
	\end{center}
\end{table} -4
\begin{table}
	\begin{center}
		\begin{tabular}{c|c}
			x & y \\
			\hline
			-5 & -10 \\
			-8 & -12 \\
			-11 & -14 \\
			-14 & -16 \\
			-17 & -18 \\
			-20 & -20 \\
			-23 & -22 \\
		\end{tabular}
	\end{center}
\end{table} 2/3
\begin{tabular}{rr}
\hline
   x &   y \\
\hline
   4 &  12 \\
   7 &  14 \\
  10 &  16 \\
  13 &  18 \\
  16 &  20 \\
\hline
\end{tabular} 2/3
\begin{tabular}{rr}
\hline
   x &   y \\


In [ ]:
import random
import sympy
import fractions

def fix_zero_neg(input):
    if int(input) == 0:
        return 0 
    else:
        return input


def AGS_Find_The_Slope_Section_3(difficulty=1, expr="latex"):
    problem = ""
    answer = ""

    half_1 = "" # SPLIT DA HALVES
    half_2 = ""

    negative = ["", "-"]
    filler = ""

    sym_1 = random.choice(["+", "-"])

    intA = fix_zero_neg(str(random.choice(negative)) + str(random.randint(1,10)))
    intB = fix_zero_neg(str(random.choice(negative)) + str(random.randint(1,10)))
    intC = fix_zero_neg(str(random.randint(2,10)))
    intD = fix_zero_neg(str(random.randint(1,10)))
    intE = fix_zero_neg(str(random.choice(negative)) + str(random.randint(1,20)))
    intF = fix_zero_neg(str(random.choice(negative)) + str(random.randint(1,20)))
    intG = fix_zero_neg(str(random.randint(2,20)))
    intH = fix_zero_neg(str(random.randint(1,20)))

    if difficulty == 1:     # Easy
        half_1 = fr'y'
        if int(intA) == 1:
            half_2 = fr'x {sym_1} {intC}'
        elif int(intA) == -1:
            half_2 = fr'- x {sym_1} {intC}'
        else:
            half_2 = fr'{intA} * x {sym_1} {intC}'

        half_1 = sympy.sympify(half_1, evaluate = False)
        half_2 = sympy.sympify(half_2, evaluate = False)

        problem = str(sympy.latex(half_1)) + " = " + str(sympy.latex(half_2, mode = "plain"))
        
        answer = "$ " + str(intA) + " $"

    elif difficulty == 2:   # Medium
        filler = random.randint(1,3)
        if filler == 1:
            half_1 = fr'y'
            half_2 = fr'{intA} / {intB} * x {sym_1} {intH}'

            half_1 = sympy.sympify(half_1, evaluate = False)
            half_2 = sympy.sympify(half_2, evaluate = False)

            problem = str(sympy.latex(half_1)) + " = " + str(sympy.latex(half_2))

            answer = "$ "+ str(sympy.latex(sympy.sympify(str(intA) + " / " + str(intB)))) + " $"
        
        elif filler == 2:
            half_1 = fr'{intG} * y'
            half_2 = fr'{intC} * x {sym_1} {intH}'

            half_1 = sympy.sympify(half_1, evaluate = False)
            half_2 = sympy.sympify(half_2, evaluate = False)

            problem = str(sympy.latex(half_1)) + " = " + str(sympy.latex(half_2))
            
            answer = "$ "+ str(sympy.latex(sympy.sympify(str(intC) + " / " + str(intG)))) + " $"
        
        else:
            half_1 = fr'{intC} * y {sym_1} {intH}'
            half_2 = fr'{intG} * x'

            half_1 = sympy.sympify(half_1, evaluate = False)
            half_2 = sympy.sympify(half_2, evaluate = False)

            problem = str(sympy.latex(half_1)) + " = " + str(sympy.latex(half_2))
            
            answer = "$ "+ str(sympy.latex(sympy.sympify(str(intG) + " / " + str(intC)))) + " $"

    elif difficulty == 3:   # Hard
        filler = random.randint(1,5)
        if filler == 1:
            y = sympy.Symbol("y")
            problem = fr'{int(intD) + 1} * x {sym_1} {intC} * y = {intA}' # intB  x ±  intA  y= intD

            half_1 = fr'{int(intD) + 1} * x {sym_1} {intC} * y'
            half_2 = fr'{intA}'

            half_1 = sympy.sympify(half_1, evaluate = False)
            half_2 = sympy.sympify(half_2, evaluate = False)

            problem = str(sympy.latex(half_1)) + " = " + str(sympy.latex(half_2))
            filler = sympy.Eq(half_1, half_2)
            filler = str(sympy.solve(filler, y))
            filler = filler[1:-1]
            filler = filler.split()
            for i in range(len(filler)):
                if filler[i].find("x") != -1:
                    answer = filler[i]
            answer = answer.replace("x", "1")
            
            answer = "$ " + str(sympy.latex(sympy.sympify(answer))) + " $"
        
        elif filler == 2:
            half_1 = fr'{intG} * x {sym_1} {intC} * y'
            half_2 = fr'{intB}'

            y = sympy.Symbol("y")
            half_1 = sympy.sympify(half_1, evaluate = False)
            half_2 = sympy.sympify(half_2, evaluate = False)

            problem = str(sympy.latex(half_1)) + " = " + str(sympy.latex(half_2))
            filler = sympy.Eq(half_1, half_2)
            filler = str(sympy.solve(filler, y))
            filler = filler[1:-1]
            filler = filler.split()
            for i in range(len(filler)):
                if filler[i].find("x") != -1:
                    answer = filler[i]
            answer = answer.replace("x", "1")
            
            answer = "$ " + str(sympy.latex(sympy.sympify(answer))) + " $"
        
        elif filler == 3:
            half_1 = fr'x / {intF} {sym_1} y / {intG}'
            half_2 = fr'1'
#intC intG
            y = sympy.Symbol("y")
            half_1 = sympy.sympify(half_1, evaluate = False)
            half_2 = sympy.sympify(half_2, evaluate = False)

            problem = str(sympy.latex(half_1)) + " = " + str(sympy.latex(half_2))
            filler = sympy.Eq(half_1, half_2)
            filler = str(sympy.solve(filler, y))
            filler = filler[1:-1]
            filler = filler.split()
            for i in range(len(filler)):
                if filler[i].find("x") != -1:
                    answer = filler[i]
            answer = answer.replace("x", "1")
            
            answer = "$ " + str(sympy.latex(sympy.sympify(answer))) + " $"
        
        elif filler == 4:
            half_1 = fr'{intH} * x {sym_1} {intG} * y + {intE}'
            half_2 = fr'0'

            y = sympy.Symbol("y")
            half_1 = sympy.sympify(half_1, evaluate = False)
            half_2 = sympy.sympify(half_2, evaluate = False)

            problem = str(sympy.latex(half_1)) + " = " + str(sympy.latex(half_2))
            filler = sympy.Eq(half_1, half_2)
            filler = str(sympy.solve(filler, y))
            filler = filler[1:-1]
            filler = filler.split()
            for i in range(len(filler)):
                if filler[i].find("x") != -1:
                    answer = filler[i]
            answer = answer.replace("x", "1")
            
            answer = "$ " + str(sympy.latex(sympy.sympify(answer))) + " $"
        else:
            if random.randint(1,2) == 1:
                problem = fr'x = {intF}' # x = intf || y=  intf
                answer = fr'Undefined'
            else:
                problem = fr'y = {intF}'
                answer = fr'0'
            answer = "$ "+ str(answer) + " $" ##

    return("$ " + str(problem) + " $", answer)

print(1)
for i in range(3):
    problem, answer = AGS_Find_The_Slope_Section_3(1, "latex")
    print(problem, answer)

print(2)
for i in range(3):
    problem, answer = AGS_Find_The_Slope_Section_3(2, "latex")
    print(problem, answer)

print(3)
for i in range(3):
    problem, answer = AGS_Find_The_Slope_Section_3(3, "latex")
    print(problem, answer)




easy
$ y = 5 x + 6 $ $ 5 $
$ y = 6 + x \left(-5\right) $ $ -5 $
$ y = x \left(-5\right) - 2 $ $ -5 $
medium
$ y = \frac{9 x}{8} - 13 $ $ \frac{9}{8} $
$ 20 y = 5 x - 10 $ $ \frac{1}{4} $
$ y = \frac{1}{4} \left(-2\right) x - 10 $ $ - \frac{1}{2} $
hard
$ \frac{x}{-6} + \frac{y}{3} = 1 $ $ \frac{1}{2} $
$ 6 x - 5 y = 7 $ $ \frac{6}{5} $
$ \frac{x}{-15} - \frac{y}{9} = 1 $ $ - \frac{3}{5} $


In [ ]:
import random
import sympy
import fractions

def fix_zero_neg(input):
    if int(input) == 0:
        return 0 
    else:
        return input


def AGS_Find_The_Slope_Section_4(difficulty=1, expr="latex"):
    rise = ""
    run = ""
    problem = ""
    answer = ""
    negative = ["", "-"]
    filler = ""

    intA = fix_zero_neg(str(random.choice(negative)) + str(random.randint(1,10)))
    intB = fix_zero_neg(str(random.choice(negative)) + str(random.randint(1,10)))
    intC = fix_zero_neg(str(random.choice(negative)) + str(random.randint(1,10)))
    intD = fix_zero_neg(str(random.choice(negative)) + str(random.randint(1,10)))
    intE = fix_zero_neg(str(random.choice(negative)) + str(random.randint(1,20)))
    intF = fix_zero_neg(str(random.choice(negative)) + str(random.randint(1,20)))
    intG = fix_zero_neg(str(random.choice(negative)) + str(random.randint(1,20)))
    intH = fix_zero_neg(str(random.choice(negative)) + str(random.randint(1,20)))

    if difficulty == 1:     # Easy
        problem = fr'( {intA} , {intB} ), ( {intC} , {intD})'
        rise = (int(intD) - int(intB))
        run = (int(intC) - int(intA))

        answer = fr'{rise} / {run}'
        answer = sympy.sympify(answer)
        if run == 0:
            answer = "Undefined"

    elif difficulty == 2 or difficulty == 3:    # Medium or hard
        filler = random.randint(1,3)
        if filler == 1:
            problem = fr'( {intE} , {intF} ), ( {intG} , {intH})'
            rise = (int(intH) - int(intF))
            run = (int(intG) - int(intE))

        elif filler == 2:
            problem = fr'( {intE} , {intF} ), ( {intG} , {intH})'
            rise = (int(intH) - int(intF))
            run = (int(intG) - int(intE))

        elif filler == 3:
            problem = fr'( {intE} , {intF} ), ( {intG} , {intF})'
            rise = (int(intF) - int(intF))
            run = (int(intG) - int(intE))

        answer = fr'{rise} / {run}'
        answer = sympy.sympify(answer)
        if run == 0:
            answer = "Undefined"

    return(sympy.latex(sympy.sympify(problem, evaluate=False)), answer)


for i in range(3):
    problem, answer = AGS_Find_The_Slope_Section_4(1, "latex")
    print(problem, answer)

for i in range(3):
    problem, answer = AGS_Find_The_Slope_Section_4(2, "latex")
    print(problem, answer)

\left( \left( -4, \  6\right), \  \left( -7, \  -9\right)\right) 5
\left( \left( 1, \  2\right), \  \left( -7, \  1\right)\right) 1/8
\left( \left( -10, \  9\right), \  \left( -7, \  5\right)\right) -4/3
\left( \left( -3, \  3\right), \  \left( 2, \  -10\right)\right) -13/5
\left( \left( -16, \  20\right), \  \left( -1, \  -8\right)\right) -28/15
\left( \left( -1, \  11\right), \  \left( 8, \  11\right)\right) 0


## 1.5.1 Information To Geometric sequence.


### section 1
#### instruction : In the next problems, you are given various types of information. Write the recursive and explicit functions for each geometric sequence. Finally, graph each sequence, making sure you clearly label the axes.
---

1. Converting word problem 
- word problem : Claire has $\$300$ in an account. She decides she is going to take out half of what’s left in there at the end of each month.
- answer : 
  - a) recurssive : 
  - b) explicit : 
  - c) table :
  - d) plot : 

2. graph notation
- problem : 2,4,6,8....
- answer : 
  - a) recurssive : 
  - b) explicit : 
  - c) table :
  - d) plot : 

3. graphical representation of the sequence. (# don't worry about this atm)
- answer : 
  - a) recurssive : 
  - b) explicit : 
  - c) table :
  - d) plot : 




In [ ]:
def infoToSeq(case=1, kind='lin', expr='latex'):
    if case == 1: # word problem
        problem, solns, labels = makeWordProb(kind, expr)
        problem += r' \newline '

        if type(solns[0]) != list:
            seq = ArithSeq(solns[0],solns[1]) if kind=='lin' else GeoSeq(solns[0],solns[1])
        else:
            seq = ArithSeq.fromPts(solns[0],solns[1]) if kind=='lin' else GeoSeq.fromPts(solns[0],solns[1])
    else:
        common = getInt(-7,7) if kind=='lin' else getInt(-5,5,exclude=[-1,0,1])
        start = getInt(-5,5)
        seq = ArithSeq(common, [1,start]) if kind=='lin' else GeoSeq(common, [1,start])
        labels = ['$n$', '$f(n)$']

        if case == 2: # list
            problem = seq.getSeqStr() + r' \newline '
        elif case == 3: # table
            problem = seq.getTable([1,2,3,4]) #+ r' \newline '
        elif case == 4: # graph
            problem = startGraph(-1,10,-10,10)
            for jj in range(1,5):
                problem += drawPt([jj,seq.findTerm(jj)])
            problem += endGraph() + r' \newline '

    answer = 'a) ' + signify(seq.getRecursive()) + r' \newline '
    answer += 'b) ' + signify(seq.getExplicit()) + r' \newline '
    if case != 3:
        problem += seq.getTable(range(1,6), vals=False, labels=labels) #+ r' \newline '
        answer += seq.getTable(range(1,6), labels=labels) #+ r' \newline '
    if case != 4:
        problem += emptyGraph(-1,10,-10,10)
        answer += startGraph(-1,10,-10,10)
        for jj in range(1,5):
            answer += drawPt([jj,seq.findTerm(jj)])
        answer += endGraph() + r' \newline '

    if expr != 'latex':   
        answer = seq

    return problem, answer

for jj in range(10):
    problem, answer = infoToSeq(case=1,kind='exp')
    print(problem)
    print(answer)

NameError: ignored

## 1.5.2 Slope & Slope Intercept Form (DONE)

### section 1
#### instruction: Use the middle column in each of the given tables to write the process or number patterns that relate the input to the output. Then create both a recursive and explicit function rule.

1) positive int slope
- A(-3,-3) and D(0,5)
- answer
  - plot : "display empty grid"
  - slope: m = 

2) negative int slope 
- E(-2,-1) and N(-4,4)
- answer
  - plot : "display empty grid"
  - slope: m = 

3) positive fraction slope
- S(0,3) and W(1,6)
- answer
  - plot : "display empty grid"
  - slope: m = 

4) negative fraction slope
- display liner equation with 2 poubts
- answer
  - plot : "display empty grid"
  - slope: m = 

5) vertical line
- answer
  - plot : "display empty grid"
  - slope: m = 

6) horizontal line
- answer
  - plot : "display empty grid"
  - slope: m = 



In [ ]:
# # latex code to plot two points


# \documentclass[letterpaper,12pt]{book}%
# \usepackage[T1]{fontenc}%
# \usepackage[utf8]{inputenc}%
# \usepackage{lmodern}%
# \usepackage{textcomp}%
# \usepackage{lastpage}%
# \usepackage{multicol}%

# \usepackage{tikz}%\usetikzlibrary{arrows}

# \newcommand{\drawpoints}[6]{
# \fill [black](#1,#2) circle(6pt) node[right]{$#5$};
# \fill [black](#3,#4) circle(6pt) node[right]{$#6$};
# }
# \begin{document}
# \begin{tikzpicture}[scale=.28,extended line/.style={shorten >=-#1,shorten <=-#1},]
# \draw [help lines] (-10,-10) grid (10,10);
# % Euclidean
# \draw [<->](0,-10.2)--(0,10.2) node[right]{$y$};
# \draw [<->](-10.2,0)--(10.2,0) node[right]{$x$};

# % draw ticks and its labels
# \foreach \x in {-10,,,,,-5,,,,,,,,5,,,,,10}
# \draw (\x,1pt) -- (\x,-3pt) node[anchor=north] {\small\x};
# \foreach \y in {-10,,,,,-5,,,,,,,,5,,,,,10}
# \draw (1pt,\y) -- (-3pt,\y) node[anchor=east] {\small\y}; 

# \drawpoints{2}{3}{4}{5}



# \end{tikzpicture}
# \end{document}

In [ ]:
import random
import sympy
import fractions


def fix_zero_neg(input):
    if int(input) == 0:
        return 0 
    else:
        return input


def linear_graph_line_formatter(slope, integer):
    output = ""
    output = r'\LinearEquation`{a}~`{b}~'.format(a = slope, b = integer)
    output = output.replace("`", "{")
    output = output.replace("~", "}")
    return output

def two_point_graph(one, one_2, two, two_2, vari_1, vari_2):
    # latex code to plot two points
    out = []
    inputted = ""
    out.append(r'\begin{tikzpicture}[scale=.28,extended line/.style={shorten >=-#1,shorten <=-#1},]')
    out.append(r'\draw [help lines] (-10,-10) grid (10,10);')
    out.append(r'\draw [<->](0,-10.2)--(0,10.2) node[right]{$y$};')
    out.append(r'\draw [<->](-10.2,0)--(10.2,0) node[right]{$x$};')

    out.append(r'\foreach \x in {-10,,,,,-5,,,,,,,,5,,,,,10}')
    out.append(r'\draw (\x,1pt) -- (\x,-3pt) node[anchor=north] {\small\x};')
    out.append(r'\foreach \y in {-10,,,,,-5,,,,,,,,5,,,,,10}')
    out.append(r'\draw (1pt,\y) -- (-3pt,\y) node[anchor=east] {\small\y};')
    inputted = fr'\drawpoints`{one}~`{one_2}~`{two}~`{two_2}~`{vari_1}~`{vari_2}~'
    inputted = inputted.replace('`', '{')
    inputted = inputted.replace('~', '}')
    out.append(inputted)

    out.append(r'\end{tikzpicture}')
    return ''.join(out)


def Graphing_Lines_Slope_Intersect_Section_1(difficulty=1, expr="latex"):
    half_1 = ""
    half_2 = ""

    problem = ""
    answer = ""
    case = random.randint(1,3)

    sym_1 = random.choice(["-", "+"])
    variables_1 = random.choice(['E', 'K', 'U', 'R', 'D', 'W', 'G', 'H', 'V'])
    variables_2 = random.choice(['X', 'Y', 'A', 'B', 'Z', 'P', 'T'])

    intA = str(random.randint(-10,10))
    intB = str(random.randint(-10,10))
    intC = str(random.randint(-10,10))
    intD = str(random.randint(-10,10))

    if case == 1:
        half_1 = fr'{variables_1}({intA},{intB})'
        half_2 = fr'{variables_2}({intC},{intD})'
        problem = "$" + half_1 + ", " + half_2 + "$"
        problem = problem + " " + str(two_point_graph(intA, intB, intC, intD, variables_1, variables_2))
        if int(intA)-int(intC) == 0:
            answer = "Undefined"
        else:
            answer = fractions.Fraction(int(intB)-int(intD),int(intA)-int(intC)) # if denom is 0, then undefine

    elif case == 2:
        half_1 = fr''
        half_2 = fr''
        
        answer = str(linear_graph_line_formatter(str(intA) + " / " + str(intB), str(sym_1) + str(intC)))
    
    elif case == 3:
        if random.randint(1,2) == 1:
            half_1 = fr'y'
            answer = "0"
        else:
            half_1 = fr'x'
            answer = "Undefined"
        half_2 = fr'{intD}'
        

        problem = str(half_1) + " = " + str(half_2)
    return(str(problem) + r"\\What is the Slope?", "Slope (m) = " + "$" + str(answer) + "$")


for i in range(3):
    problem, answer = Graphing_Lines_Slope_Intersect_Section_1(1, "latex")
    print(problem, answer)


\\What is the Slope? Slope (m) = $\LinearEquation{6 / 2}{+2}$
x = -1\\What is the Slope? Slope (m) = $Undefined$
$E(-7,5), P(-6,-10)$ \begin{tikzpicture}[scale=.28,extended line/.style={shorten >=-#1,shorten <=-#1},]\draw [help lines] (-10,-10) grid (10,10);\draw [<->](0,-10.2)--(0,10.2) node[right]{$y$};\draw [<->](-10.2,0)--(10.2,0) node[right]{$x$};\foreach \x in {-10,,,,,-5,,,,,,,,5,,,,,10}\draw (\x,1pt) -- (\x,-3pt) node[anchor=north] {\small\x};\foreach \y in {-10,,,,,-5,,,,,,,,5,,,,,10}\draw (1pt,\y) -- (-3pt,\y) node[anchor=east] {\small\y};\drawpoints{-7}{5}{-6}{-10}{E}{P}\end{tikzpicture}\\What is the Slope? Slope (m) = $-15$


### section 2
#### Instruction : Find the slope, $m$ , and the y-intercept, $b$ , for each line. Then write an equation for the line. $ y = mx + b $

1) positive int slope
- answer
  - a) m = 
  - b) b = 
  - c) Equation = 

2) negative int slope 
- answer
  - a) m = 
  - b) b = 
  - c) Equation = 

3) positive fraction slope
- answer
  - a) m = 
  - b) b = 
  - c) Equation = 

4) negative fraction slope
- answer
  - a) m = 
  - b) b = 
  - c) Equation = 

5) vertical line
- answer
  - a) m = 
  - b) b = 
  - c) Equation = 

6) horizontal line
- answer
  - a) m = 
  - b) b = 
  - c) Equation = 

In [ ]:
import random
import sympy
import fractions


def fix_zero_neg(input):
    if int(input) == 0:
        return 0 
    else:
        return input


def linear_graph_line_formatter(slope, integer):
    output = ""
    output = r'\LinearEquation`{a}~`{b}~'.format(a = slope, b = integer)
    output = output.replace("`", "{")
    output = output.replace("~", "}")
    return output


def Graphing_Lines_Slope_Intersect_Section_1(difficulty=1, expr="latex"):
    half_1 = ""
    half_2 = ""

    problem = ""
    answer = ""
    negative = ["", "-"]

    sym_1 = random.choice(["-", "+"])
    
    intA = fix_zero_neg(str(random.choice(negative)) + str(random.randint(2,10)))
    intB = fix_zero_neg(str(random.randint(1,10)))
    intC = fix_zero_neg(str(random.randint(1,10)))
    intD = fix_zero_neg(str(random.choice(negative)) + str(random.randint(1,10)))

    if difficulty == 1:     # Easy
        half_1 = fr'y'
        half_2 = fr'{intA} x {sym_1} {intB}'

        problem = str(half_1) + " = " + str(half_2)
        if sym_1 == "+":
            sym_1 = ""
        answer = str(linear_graph_line_formatter(intA, str(sym_1) + str(intB)))

    elif difficulty == 2:   # Medium
        half_1 = fr'y'
        half_2 = fr'\frac`{intA}~`{intB}~ x {sym_1} {intC}'

        half_2 = half_2.replace("`", "{")
        half_2 = half_2.replace("~", "}")
        
        problem = str(half_1) + " = " + str(half_2)
        if sym_1 == "+":
            sym_1 = ""
        
        answer = str(linear_graph_line_formatter(str(intA) + " / " + str(intB), str(sym_1) + str(intC)))
    
    elif difficulty == 3:   # Hard
        if random.randint(1,2) == 1:
            half_1 = fr'y'
            answer = "0"
        else:
            half_1 = fr'x'
            answer = "Undefined"
        half_2 = fr'{intD}'
        

        problem = str(half_1) + " = " + str(half_2)
    return("$ " + str(problem) + " $", answer)


for i in range(3):
    problem, answer = Graphing_Lines_Slope_Intersect_Section_1(1, "latex")
    print(problem, answer)

for i in range(3):
    problem, answer = Graphing_Lines_Slope_Intersect_Section_1(2, "latex")
    print(problem, answer)

for i in range(3):
    problem, answer = Graphing_Lines_Slope_Intersect_Section_1(3, "latex")
    print(problem, answer)

$ y = 4 x - 2 $ \LinearEquation{4}{-2}
$ y = -7 x + 9 $ \LinearEquation{-7}{9}
$ y = -9 x + 9 $ \LinearEquation{-9}{9}
$ y = \frac{-5}{1} x - 2 $ \LinearEquation{-5 / 1}{-2}
$ y = \frac{-4}{6} x - 1 $ \LinearEquation{-4 / 6}{-1}
$ y = \frac{6}{8} x - 3 $ \LinearEquation{6 / 8}{-3}
$ y = -3 $ 0
$ y = -9 $ 0
$ x = -2 $ Undefined


In [ ]:
import random
import sympy
import fractions

def fix_zero_neg(input):
    if int(input) == 0:
        return 0 
    else:
        return input


def Graphing_Lines_Slope_Intersect_Section_2(difficulty=1, expr="latex"):
    problem = ""
    answer = ""

    half_1 = "" # SPLIT DA HALVES
    half_2 = ""

    negative = ["", "-"]
    filler = ""

    sym_1 = random.choice(["+", "-"])

    intA = fix_zero_neg(str(random.choice(negative)) + str(random.randint(1,10)))
    intB = fix_zero_neg(str(random.choice(negative)) + str(random.randint(1,10)))
    intC = fix_zero_neg(str(random.randint(2,10)))
    intD = fix_zero_neg(str(random.randint(1,10)))
    intE = fix_zero_neg(str(random.choice(negative)) + str(random.randint(1,20)))
    intF = fix_zero_neg(str(random.choice(negative)) + str(random.randint(1,20)))
    intG = fix_zero_neg(str(random.randint(2,20)))
    intH = fix_zero_neg(str(random.randint(1,20)))

    if difficulty == 1:     # Easy
        half_1 = fr'y'
        if int(intA) == 1:
            half_2 = fr'x {sym_1} {intC}'
        elif int(intA) == -1:
            half_2 = fr'- x {sym_1} {intC}'
        else:
            half_2 = fr'{intA} * x {sym_1} {intC}'

        half_1 = sympy.sympify(half_1, evaluate = False)
        half_2 = sympy.sympify(half_2, evaluate = False)

        problem = str(sympy.latex(half_1)) + " = " + str(sympy.latex(half_2, mode = "plain"))
        
        answer = "m = " + "$ " + str(intA) + " $", "b = " + "$ " + str(intC) + " $" 

    elif difficulty == 2:   # Medium
        filler = random.randint(1,3)
        if filler == 1:
            half_1 = fr'y'
            half_2 = fr'{intA} / {intB} * x {sym_1} {intH}'

            half_1 = sympy.sympify(half_1, evaluate = False)
            half_2 = sympy.sympify(half_2, evaluate = False)

            problem = str(sympy.latex(half_1)) + " = " + str(sympy.latex(half_2))

            answer = "$ "+ str(sympy.latex(sympy.sympify(str(intA) + " / " + str(intB)))) + " $"
        
        elif filler == 2:
            half_1 = fr'{intG} * y'
            half_2 = fr'{intC} * x {sym_1} {intH}'

            half_1 = sympy.sympify(half_1, evaluate = False)
            half_2 = sympy.sympify(half_2, evaluate = False)

            problem = str(sympy.latex(half_1)) + " = " + str(sympy.latex(half_2))
            
            answer = "$ "+ str(sympy.latex(sympy.sympify(str(intC) + " / " + str(intG)))) + " $"
        
        else:
            half_1 = fr'{intC} * y {sym_1} {intH}'
            half_2 = fr'{intG} * x'

            half_1 = sympy.sympify(half_1, evaluate = False)
            half_2 = sympy.sympify(half_2, evaluate = False)

            problem = str(sympy.latex(half_1)) + " = " + str(sympy.latex(half_2))
            
            answer = "$ "+ str(sympy.latex(sympy.sympify(str(intG) + " / " + str(intC)))) + " $"

    elif difficulty == 3:   # Hard
        filler = random.randint(1,5)
        if filler == 1:
            y = sympy.Symbol("y")
            problem = fr'{int(intD) + 1} * x {sym_1} {intC} * y = {intA}' # intB  x ±  intA  y= intD

            half_1 = fr'{int(intD) + 1} * x {sym_1} {intC} * y'
            half_2 = fr'{intA}'

            half_1 = sympy.sympify(half_1, evaluate = False)
            half_2 = sympy.sympify(half_2, evaluate = False)

            problem = str(sympy.latex(half_1)) + " = " + str(sympy.latex(half_2))
            filler = sympy.Eq(half_1, half_2)
            filler = str(sympy.solve(filler, y))
            filler = filler[1:-1]
            filler = filler.split()
            for i in range(len(filler)):
                if filler[i].find("x") != -1:
                    answer = filler[i]
            answer = answer.replace("x", "1")
            
            answer = "$ " + str(sympy.latex(sympy.sympify(answer))) + " $"
        
        elif filler == 2:
            half_1 = fr'{intG} * x {sym_1} {intC} * y'
            half_2 = fr'{intB}'

            y = sympy.Symbol("y")
            half_1 = sympy.sympify(half_1, evaluate = False)
            half_2 = sympy.sympify(half_2, evaluate = False)

            problem = str(sympy.latex(half_1)) + " = " + str(sympy.latex(half_2))
            filler = sympy.Eq(half_1, half_2)
            filler = str(sympy.solve(filler, y))
            filler = filler[1:-1]
            filler = filler.split()
            for i in range(len(filler)):
                if filler[i].find("x") != -1:
                    answer = filler[i]
            answer = answer.replace("x", "1")
            
            answer = "$ " + str(sympy.latex(sympy.sympify(answer))) + " $"
        
        elif filler == 3:
            half_1 = fr'x / {intF} {sym_1} y / {intG}'
            half_2 = fr'1'
#intC intG
            y = sympy.Symbol("y")
            half_1 = sympy.sympify(half_1, evaluate = False)
            half_2 = sympy.sympify(half_2, evaluate = False)

            problem = str(sympy.latex(half_1)) + " = " + str(sympy.latex(half_2))
            filler = sympy.Eq(half_1, half_2)
            filler = str(sympy.solve(filler, y))
            filler = filler[1:-1]
            filler = filler.split()
            for i in range(len(filler)):
                if filler[i].find("x") != -1:
                    answer = filler[i]
            answer = answer.replace("x", "1")
            
            answer = "$ " + str(sympy.latex(sympy.sympify(answer))) + " $"
        
        elif filler == 4:
            half_1 = fr'{intH} * x {sym_1} {intG} * y + {intE}'
            half_2 = fr'0'

            y = sympy.Symbol("y")
            half_1 = sympy.sympify(half_1, evaluate = False)
            half_2 = sympy.sympify(half_2, evaluate = False)

            problem = str(sympy.latex(half_1)) + " = " + str(sympy.latex(half_2))
            filler = sympy.Eq(half_1, half_2)
            filler = str(sympy.solve(filler, y))
            filler = filler[1:-1]
            filler = filler.split()
            for i in range(len(filler)):
                if filler[i].find("x") != -1:
                    answer = filler[i]
            answer = answer.replace("x", "1")
            
            answer = "$ " + str(sympy.latex(sympy.sympify(answer))) + " $"
        else:
            if random.randint(1,2) == 1:
                problem = fr'x = {intF}' # x = intf || y=  intf
                answer = fr'Undefined'
            else:
                problem = fr'y = {intF}'
                answer = fr'0'
            answer = "$ "+ str(answer) + " $" ##

    return("$ " + str(problem) + " $", answer)


for i in range(3):
    problem, answer = Graphing_Lines_Slope_Intersect_Section_2(1, "latex")
    print(problem, answer)

for i in range(3):
    problem, answer = Graphing_Lines_Slope_Intersect_Section_2(2, "latex")
    print(problem, answer)

for i in range(3):
    problem, answer = Graphing_Lines_Slope_Intersect_Section_2(3, "latex")
    print(problem, answer)

$ y = x \left(-2\right) - 9 $ ('m = $ -2 $', 'b = $ 9 $')
$ y = 9 + x \left(-7\right) $ ('m = $ -7 $', 'b = $ 9 $')
$ y = 3 x + 4 $ ('m = $ 3 $', 'b = $ 4 $')
$ 3 y - 3 = 2 x $ $ \frac{2}{3} $
$ 18 y = 4 x - 5 $ $ \frac{2}{9} $
$ 2 y = 8 x + 2 $ $ 4 $
$ \frac{x}{-17} + \frac{y}{2} = 1 $ $ \frac{2}{17} $
$ \frac{x}{11} + \frac{y}{7} = 1 $ $ \frac{7}{11} $
$ \frac{x}{5} + \frac{y}{9} = 1 $ $ \frac{9}{5} $


## 1.5.3. Geometric Sequence To Explicit & Recurssive (REPEAT OF 1.5.1?)
 

### section 1
#### instruction : In the next problems, you are given various types of information. Write the recursive and explicit functions for each geometric sequence. Finally, graph each sequence, making sure you clearly label the axes.

1) number sequence
- problem ; 2,4,8,16

Answer 
- a) Recurssive :
- b) Explicit : 
- c) Plot : 

2) Word Problems:
- problem :
Claire has $300$ in an account. She decides she is going to take out half of what’s left in there at the end of each month.

Answer 
- a) Recurssive :
- b) Explicit : 
- c) Plot : 



3) geometric sequence
- problem : 

Answer 
- a) Recurssive :
- b) Explicit : 
- c) Plot : 


4) graph sequence
- problem : 

Answer 
- a) Recurssive :
- b) Explicit : 
- c) Plot : 




## 1.6.1 Percent Change (part of prealg?) (DONE)


11% increase
1.11


40 % decrease 
0.6

In [ ]:
import sympy
from fractions import Fraction
import fractions
import random
import math
"""A LOT FOF MODULE 8
1.6.1 Percent Change(part of prealg?)
11 % increase 1.11
40 % decrease 0.6"""

# First Section
def Percents_Fractions_Decimal_Conversion_1(difficulty= 1, expr="latex"):
    problemsets = []
    answerset = []
    temp = 0
    list_temp = []

    if(difficulty == 1):        # Easy
        problemsets.append("0." + str(random.randint(1, 99)))

    elif(difficulty == 2):      # Medium
        problemsets.append(str(random.randint(1, 99)) +
                           "." + str(random.randint(1, 99)))

    elif(difficulty == 3):      # Hard
        problemsets.append(str(random.randint(-999, 999)) +
                           "." + str(random.randint(1, 999)))

    temp = round(float(str(problemsets[0])) * 100, 2)
    list_temp = list(str(temp))
    if int(list_temp[-1]) == 0:
        temp = int(temp)
    answerset.append(str(temp) + "%")

    return(answerset[0], problemsets[0])

# Second Section
def Percents_Fractions_Decimal_Conversion_2(difficulty = 1, option_random = "latex"):
    problemsets = []
    answerset = []
    temp = 0
    list_temp = []

    if(difficulty == 1):        # Easy
        problemsets.append("0." + str(random.randint(1, 99)))

    elif(difficulty == 2):      # Medium
        problemsets.append(str(random.randint(1, 20)) +
                           "." + str(random.randint(1, 99)))

    elif(difficulty == 3):      # Hard
        problemsets.append(str(random.randint(1, 999)) +
                           "." + str(random.randint(1, 999)))

    temp = round(float(str(problemsets[0])) * 100, 2)
    list_temp = list(str(temp))
    if int(list_temp[-1]) == 0:
        temp = int(temp)
    answerset.append(str(temp) + "%")
    return(problemsets[0], answerset[0])


# Third Section NEED ALTERNATE TO FRACTION
def Percents_Fractions_Decimal_Conversion_3(difficulty=1, option_random="latex"):
    problemsets_1 = []
    problemsets_2 = []
    answerset = []
    temp = 0
    list_temp = []
    percent = 0
    replace = ""
    holder = 0

    if(difficulty == 1):        # Easy
        problemsets_2.append("0." + str(random.randint(1, 99)))

    elif(difficulty == 2):      # Medium
        problemsets_2.append("0." + str(random.randint(1, 999)))

    elif(difficulty == 3):      # Hard
        problemsets_2.append("0." + str(random.randint(1, 9999)))

    temp = round(float(str(problemsets_2[0])) * 100, 2)

    list_temp = list(str(temp))
    if int(list_temp[-1]) == 0:
        temp = int(temp)
    problemsets_1.append(str(temp) + "%")

    percent = Fraction(problemsets_2[0])
    percent = str(percent)
    percent.replace("Fraction(", '')
    percent.replace(")", '')
    answerset.append(percent)
    return(problemsets_1[0], answerset[0])


# Fourth Section
def Percents_Fractions_Decimal_Conversion_4(difficulty = 1, option_random="latex"):
    problemsets_1 = []
    problemsets_2 = []
    answerset = []
    temp = 0
    list_temp = []
    percent = 0
    replace = ""
    holder = 0

    if(difficulty == 1):        # Easy
        problemsets_2.append("0." + str(random.randint(1, 99)))

    elif(difficulty == 2):      # Medium
        problemsets_2.append("0." + str(random.randint(1, 999)))

    elif(difficulty == 3):      # Hard
        problemsets_2.append("0." + str(random.randint(1, 9999)))

    temp = round(float(str(problemsets_2[0])) * 100, 2)

    list_temp = list(str(temp))
    if int(list_temp[-1]) == 0:
        temp = int(temp)
    problemsets_1.append(str(temp) + "%")

    percent = Fraction(problemsets_2[0])
    percent = str(percent)
    percent.replace("Fraction(", '')
    percent.replace(")", '')
    answerset.append(percent)
    return(answerset[0], problemsets_1[0])

def percentify(num):
    if str(float(num))[-2:] == ".0":
        num = int(num)
    return(str(num) + "%")


def inc_dec_formatter(num, choice):
    num = str(num)
    if choice == True:
        return(num + " Increase")
    elif choice == False:
        return(num + " Decrease")

measurements = ["miles", "grams", "ft", "minutes", "seconds",
                "meters", "kilos", "dollars", "liters", "tons"]
pick = [True, False]
def Finding_Percent_Change_Section_1(difficulty=1, expr='latex'):
    problem = ""
    answer = 0
    type = ""

    intA = random.randint(1, 100)
    intB = random.randint(1, 200)
    floatA = float(str(random.randint(1, 99)) +
                   "." + str(random.randint(1, 9)))
    floatB = float(str(random.randint(1, 199)) +
                   "." + str(random.randint(1, 9)))

    chosen = random.choice(pick)
    if difficulty == 1:  # From 82 to 38
        type = False
        problem = "From " + str(intA) + " to " + str(intB)
        answer = (intB - intA) / intA
        if intB >= intA:
            answer = answer + 1
            type = True
        else:
            answer = float(answer) * -1
        return(problem, inc_dec_formatter(percentify(round(100 * round(answer, 4), 2)), type))

    elif difficulty == 2:  # What percent of  81.88  is  122 ?
        if chosen == True:
            type = False
            problem = "From " + str(intA) + " to " + str(floatB)
            answer = (floatB - intA) / intA
            if floatB >= intA:
                answer = answer + 1
                type = True
            else:
                answer = float(answer) * -1
            return(problem, inc_dec_formatter(percentify(round(100 * round(answer, 4), 2)), type))
        else:
            type = False
            problem = "From " + str(floatB) + " to " + str(intB)
            answer = (intB - floatB) / floatB
            if intB >= floatB:
                answer = answer + 1
                type = True
            else:
                answer = float(answer) * -1
            return(problem, inc_dec_formatter(percentify(round(100 * round(answer, 4), 2)), type))

    elif difficulty == 3:  # 76% of what is 188.12 ? || 144.26  is  23%  of what?
        if chosen == True:
            type = False
            problem = "From " + str(floatA) + " to " + str(floatB)
            answer = (floatB - floatA) / floatA
            if floatB >= floatA:
                answer = answer + 1
                type = True
            else:
                answer = float(answer) * -1
            return(problem, inc_dec_formatter(percentify(round(100 * round(answer, 4), 2)), type))
        else:
            type = False
            problem = "From " + str(floatB) + " to " + str(floatA)
            answer = (floatA - floatB) / floatB
            if floatA >= floatB:
                answer = answer + 1
                type = True
            else:
                answer = float(answer) * -1
            return(problem, inc_dec_formatter(percentify(round(100 * round(answer, 4), 2)), type))


def Finding_Percent_Change_Section_2(difficulty=1, expr='latex'):
    problem = ""
    answer = 0
    type = ""
    measure = random.choice(measurements)

    intA = random.randint(1, 100)
    intB = random.randint(1, 200)
    floatA = float(str(random.randint(1, 99)) +
                   "." + str(random.randint(1, 9)))
    floatB = float(str(random.randint(1, 199)) +
                   "." + str(random.randint(1, 9)))

    chosen = random.choice(pick)
    if difficulty == 1:  # From 82 to 38
        type = False
        problem = "From " + str(intA) + " " + str(measure) + \
            " to " + str(intB) + " " + str(measure)
        answer = (intB - intA) / intA
        if intB >= intA:
            answer = answer + 1
            type = True
        else:
            answer = float(answer) * -1
        return(problem, inc_dec_formatter(percentify(round(100 * round(answer, 4), 2)), type))

    elif difficulty == 2:  # What percent of  81.88  is  122 ?
        if chosen == True:
            type = False
            problem = "From " + str(intA) + " " + str(measure) + \
                " to " + str(floatB) + " " + str(measure)
            answer = (floatB - intA) / intA
            if floatB >= intA:
                answer = answer + 1
                type = True
            else:
                answer = float(answer) * -1
            return(problem, inc_dec_formatter(percentify(round(100 * round(answer, 4), 2)), type))
        else:
            type = False
            problem = "From " + \
                str(floatB) + " " + str(measure) + \
                " to " + str(intB) + " " + str(measure)
            answer = (intB - floatB) / floatB
            if intB >= floatB:
                answer = answer + 1
                type = True
            else:
                answer = float(answer) * -1
            return(problem, inc_dec_formatter(percentify(round(100 * round(answer, 4), 2)), type))

    elif difficulty == 3:  # 76% of what is 188.12 ? || 144.26  is  23%  of what?
        if chosen == True:
            type = False
            problem = "From " + \
                str(floatA) + " " + str(measure) + " to " + \
                str(floatB) + " " + str(measure)
            answer = (floatB - floatA) / floatA
            if floatB >= floatA:
                answer = answer + 1
                type = True
            else:
                answer = float(answer) * -1
            return(problem, inc_dec_formatter(percentify(round(100 * round(answer, 4), 2)), type))
        else:
            type = False
            problem = "From " + \
                str(floatB) + " " + str(measure) + " to " + \
                str(floatA) + " " + str(measure)
            answer = (floatA - floatB) / floatB
            if floatA >= floatB:
                answer = answer + 1
                type = True
            else:
                answer = float(answer) * -1
            return(problem, inc_dec_formatter(percentify(round(100 * round(answer, 4), 2)), type))


def money_truncate(n):
    return("{:,.2f}".format(float(n)))


def money_formatter(sent):
    return("\$" + str(sent))


def percentify(sent):
    return str(sent) + "\%"

# COMBINE EASY AND HARD
def Markup_Discount_and_Tax(difficulty=1, expr='latex'):
    intA = random.randint(1, 90)
    intB = random.randint(1, 90)
    decimal = str(random.randint(1, 99))
    if int(decimal) < 10:
        decimal = "0"+str(decimal)
    floatA = str(random.randint(10, 99)) + "." + str(decimal)

    decimal_2 = str(random.randint(1, 99))
    if int(decimal_2) < 10:
        decimal_2 = "0"+str(decimal_2)
    floatB = str(random.randint(1000, 1999)) + "." + str(decimal_2)

    cheap_items = ["a CD", "a Vinyl CD", "a desk", "a game", "a book", "a keyboard", "a notebook", "a mouse", "a pack of batteries", "a bag of chips",
                   "a box of medicine", "a pair of earbuds", "a camera", "a movie ticket", 'a DVD', 'a comic book', 'an oil change', 'a pen', 'a book', 'a camera', 'an airpod']
    expensive_items = ['a computer', 'a macbook', 'a telescope',
                       'a camera', 'a snow board', 'concert tickets', 'a telescope']

    action_up = ['Mark up', 'Tax']
    action_down = ['Sale', 'Discount']

    answer = ""
    case = random.randint(1, 2)
    sentence = ""
    if difficulty == 1:     # Easy
        """Cost of [cheap_items] :  $  floatA
                [action_up] : intA %
                x = floatA *intA * 100
                Cost of a comic book: $$$ 99.95
                Markup : 95%
                Answer :  $194.03"""
        if case == 1:
            sentence = "The cost of " + str(random.choice(cheap_items)) + ": \$" + money_truncate(
                floatA) + r"\\" + str(random.choice(action_up)) + " : " + percentify(intA)
            answer = money_formatter(money_truncate(
                (float(floatA) * float("0." + str(intA))) + float(floatA)))

        else:
            sentence = "Cost of " + str(random.choice(cheap_items)) + ": \$" + money_truncate(
                floatA) + r"\\" + str(random.choice(action_down)) + " : " + percentify(intA)
            answer = money_formatter(money_truncate(
                float(floatA) - (float(floatA) * float("0." + str(intA)))))

    elif difficulty == 2:   # Medium
        """Cost of [items] :  $  floatA
                [action_down] : intA %
                x = floatA *(1-intA) * 100
                Cost of an oil change: $$$ 1237.51
                Discount : 18%
                Answer :  $1,014.76"""
        if case == 1:
            sentence = "The cost of " + str(random.choice(expensive_items)) + ": \$" + str(
                money_truncate(floatB)) + r"\\" + str(random.choice(action_up)) + " : " + percentify(intB)
            answer = money_formatter(money_truncate(
                (float(floatB) * float("0." + str(intB))) + float(floatB)))

        else:
            sentence = "Cost of " + str(random.choice(expensive_items)) + ": \$" + str(
                money_truncate(floatB)) + r"\\" + str(random.choice(action_down)) + " : " + percentify(intB)
            answer = money_formatter(money_truncate(
                float(floatB) - (float(floatB) * float("0." + str(intB)))))

    elif difficulty == 3:   # Hard
        """  - case 1
        - Cost of [cheap_items] : $\$$ floatA
        - [action_up] : intA %
        - [action_down] : intB %

        - case 2
        - Cost of [expensive_items] : $\$$ floatA
        - [action_up] : intA %
        - [action_down] : intB %"""
        if case == 1:
            sentence = "The cost of " + str(random.choice(cheap_items)) + ": \$" + str(money_truncate(floatA)) + r"\\" + str(
                random.choice(action_up)) + " : " + percentify(intA) + ", " + str(random.choice(action_down)) + " : " + percentify(intB)
            up = 0
            up = (float(floatA) * float("0." + str(intA))) + float(floatA)
            answer = money_formatter(money_truncate(
                float(up) - (float(up) * float("0." + str(intB)))))

        else:
            sentence = "The cost of " + str(random.choice(expensive_items)) + ": \$" + str(money_truncate(floatB)) + r"\\" + str(
                random.choice(action_down)) + " : " + percentify(intB) + ", " + str(random.choice(action_up)) + " : " + percentify(intA)
            down = 0
            down = float(floatB) - (float(floatB) * float("0." + str(intB)))
            answer = money_formatter(money_truncate(
                (float(down) * float("0." + str(intA))) + float(down)))

    return(sentence, answer)

def AGS_Percent_Change(difficulty = 1, expr = "latex"):
    return random.choice([Percents_Fractions_Decimal_Conversion_1(difficulty, expr), Percents_Fractions_Decimal_Conversion_2(difficulty, expr), Percents_Fractions_Decimal_Conversion_3(difficulty, expr), Percents_Fractions_Decimal_Conversion_4(difficulty, expr), Finding_Percent_Change_Section_1(difficulty, expr), Finding_Percent_Change_Section_2(difficulty, expr), Markup_Discount_and_Tax(difficulty, expr)])
a,b = AGS_Percent_Change(1, "latex")
print(a,b)

Cost of a pair of earbuds: \$51.46\\Discount : 67\% \$16.98


##1.6.2 Is It Arithmetic Or Geometric Sequence


### section1:
#### instruction: Find the missing values for each arithmetic or geometric sequence. Select whether it has a constant difference or a constant ratio. State the value of the constant difference or ratio. Indicate if the sequence is arithmetic or geometric by selecting the correct answer.

1) Arithmatic sequence

- a) 5,10,15,$____$,25,30,$____$,....
- b) Arithmatic or geometric? Arith
- c) Common Diference or Ratio? +5
- d) Explicit?
- e) Recurssive?

2) Geometric sequence
- a) 20,10,$____$,2.5,$____$,....
- b) Arithmatic or geometric? 
- c) Common Diference or Ratio? 
- d) Explicit?
- e) Recurssive?

3) Arithmatic word problems

- a) 
- b) Arithmatic or geometric? 
- c) Common Diference or Ratio? 
- d) Explicit?
- e) Recurssive?

4) Geometric word problems

- a) 
- b) Arithmatic or geometric? 
- c) Common Diference or Ratio? 
- d) Explicit?
- e) Recurssive?




In [ ]:
def arithOrGeo_1_6_2(case=1, expr="latex"):
    if case == 1:           # arithmetic list
        seq = ArithSeq(getInt(-10,10), [1,getInt(-10,10)])
        problem = signify(seq.getSeqStr([1,2,3,'',5,6,'']))
    elif case == 2:         # geometric list
        mult = getInt(-5,5,exclude=[-1,0,1])
        ratio, startnum = [mult, 1] if randint(0,1) else [Rational(1,mult), 5]
        seq = GeoSeq(ratio, [startnum,getInt(-5,5)])
        problem = signify(seq.getSeqStr([1,2,'',4,'']))
    else:         # word problems (case 3=arithmetic, case 4=geometric)
        problem, solns, _ = makeWordProb('lin', expr) if case==3 else makeWordProb('exp', expr)
        if type(solns[0]) != list:
            seq = ArithSeq(solns[0],solns[1]) if case==3 else GeoSeq(solns[0],solns[1])
        else:
            seq = ArithSeq.fromPts(solns[0],solns[1]) if case==3 else GeoSeq.fromPts(solns[0],solns[1])

    answer = '(a) ' + signify(seq.getSeqStr(range(1,8))) + r' \newline (b) '
    answer += 'Arithmetic' if case in [1,3] else 'Geometric'
    answer += r' \newline (c) Common '
    answer += 'difference' if case in [1,3] else 'ratio'
    answer += fr' $={latexify(seq.common,seq.precision)}$' + r'\newline (d) ' + signify(seq.getExplicit())
    answer += r' \newline (e) ' + signify(seq.getRecursive())

    return problem, answer

for jj in range(10):
    problem, answer = arithOrGeo_1_6_2(randint(1,4))
    print(problem, r'\\')
    print(answer, r'\\ \\')

$1,\ 3,\ 5,\ \underline{\hspace{4mm}},\ 9,\ 11,\ \underline{\hspace{4mm}}, \ldots$ \\
(a) $1,\ 3,\ 5,\ 7,\ 9,\ 11,\ 13, \ldots$ \newline (b) Arithmetic \newline (c) Common difference $=2$\newline (d) $f(n) = 2\left(n-1\right) + 1$ \newline (e) $f(n) = f(n-1) + 2,\quad f(1) = 1$ \\ \\


NameError: ignored

##1.8.1 Fill the gap


### section 1
Each of the tables given represents a sequence. Find the missing terms in the sequence, show your method, and create the explicit and recursive equations. \\
Case 1: Skip one \\
Case 2: Skip two \\
Case 3: Skip two, blank end



In [ ]:
def FillInTheGap(kind='lin', expr='latex'):
    if kind == 'lin':
        seq = ArithSeq(getInt(-10,10), [1,getInt(-10,10)])
    else:
        seq = GeoSeq(getInt(-10,10), [1,getInt(-10,10)])

    case = randint(1,3)
    length = case + 3
    if case == 1:
        problem = seq.getTable(range(1,length), vals=[1,0,1], labels=['$x$','$y$'])
    elif case == 2:
        problem = seq.getTable(range(1,length), vals=[1,0,0,1], labels=['$x$','$y$'])
    else:
        problem = seq.getTable(range(1,length), vals=[1,0,0,1,0], labels=['$x$','$y$'])

    answer = seq.getTable(range(1,length), labels=['$x$','$y$'])
    if expr == 'latex':
        answer += signify(seq.getExplicit()) + r' \newline '
        answer += signify(seq.getRecursive())
    else:
        answer = [answer] + [seq]

    return problem, answer

for jj in range(10):
    problem, answer = FillInTheGap()
    print(problem)
    print(answer, r'\\ \\')

{\renewcommand{\arraystretch}{1.5}	\begin{center}
		\begin{tabular}{|c|c|}
			\hline
			$x$ & $y$ \\
			\hline
			$1$ & $9$ \\
			\hline
			$2$ & \phantom{$19$} \\
			\hline
			$3$ & \phantom{$29$} \\
			\hline
			$4$ & $39$ \\
			\hline
			$5$ & \phantom{$49$} \\
			\hline
		\end{tabular}
	\end{center}}
{\renewcommand{\arraystretch}{1.5}	\begin{center}
		\begin{tabular}{|c|c|}
			\hline
			$x$ & $y$ \\
			\hline
			$1$ & $9$ \\
			\hline
			$2$ & $19$ \\
			\hline
			$3$ & $29$ \\
			\hline
			$4$ & $39$ \\
			\hline
			$5$ & $49$ \\
			\hline
		\end{tabular}
	\end{center}}$f(n) = 10\left(n-1\right) + 9$ \newline $f(n) = f(n-1) + 10,\quad f(1) = 9$ \\ \\
{\renewcommand{\arraystretch}{1.5}	\begin{center}
		\begin{tabular}{|c|c|}
			\hline
			$x$ & $y$ \\
			\hline
			$1$ & $-10$ \\
			\hline
			$2$ & \phantom{$-9$} \\
			\hline
			$3$ & \phantom{$-8$} \\
			\hline
			$4$ & $-7$ \\
			\hline
		\end{tabular}
	\end{center}}
{\renewcommand{\arraystretch}{1.5}	\begin{center}
		\begin{tabula

## 1.9.1 Which Grows Faster?

### section 1 
#### instruction : find the first five terms. Then compare the growth of the arithmetic sequence and the geometric sequence. Which grows faster? When?

1) case 1: same start same diff & r
- problem: 
- Arithmetic sequence: $f(1)=3$, common difference $d=2$
- Geometric sequence: $g(1)=3$, common ratio $r=2$
- answer:
- 1) find the first five terms:
- 2) which grows faster? $f(100)=?$ or $g(100)=?$

2) case 2:
- problem:
- Arithmetic sequence: $f(n)=4+10(n-1)$
- Geometric sequence: $g(n)=256\left(\frac{1}{2}\right)^{n}$
- answer:
- 1) find the first five terms:
- 2) which grows faster? $f(100)=?$ or $g(100)=?$



In [ ]:
def Compare_the_Growth_1_9_1(case=1, expr='latex'):
    if case == 1: # Same start and common w/ growth
        start, common = randint(1,5), randint(2,5)
        arith, geo = ArithSeq(common,[1,start]), GeoSeq(common,[1,start], label='g')

        answer = r'{\bf b. }' + fr'${geo.label}(100)>{arith.label}(100)$: geometric growth always outgrows arithmetic growth eventually.'
    elif case == 2: # Arith: small start w/ growth, Geo: big start w/ decay
        arith = ArithSeq(randint(5,15), randint(1,4))
        geo = GeoSeq(Rational(1,randint(2,5)), [5,randint(1,5)], label='g')

        answer = r'{\bf b. }' + fr'${arith.label}(100)>{geo.label}(100)$: arithmetic growth will eventually exceed geometric decay.'
    elif case == 3: # Arith: big start w/ growth, Geo: small start w/ growth
        arith = ArithSeq(randint(5,15), [1,randint(200,300)])
        geo = GeoSeq(randint(2,5), [1,randint(1,5)], label='g')

        answer = r'{\bf b. }' + fr'${geo.label}(100)>{arith.label}(100)$: geometric growth always outgrows arithmetic growth eventually.'
    elif case == 4: # Arith: big start w/ decay, Geo: small start w/ growth
        arith = ArithSeq(randint(-15,-5), [1,randint(100,200)])
        geo = GeoSeq(randint(2,5), randint(1,5), label='g')

        answer = r'{\bf b. }' + fr'${geo.label}(100)>{arith.label}(100)$: geometric growth will eventually exceed arithmetic decay.'
    elif case == 5: # Same (big) start and (similar) common w/ decay
        geo = GeoSeq(Rational(1,randint(2,5)), [5,randint(1,5)], label='g')
        arith = ArithSeq(-1/geo.common, [1,geo.start])

        answer = r'{\bf b. }' + fr'${geo.label}(100)>{arith.label}(100)$: arithmetic decay eventually becomes negative while geometric decay remains positive.'


    info = randint(1,3)
    if info == 1:
        arithInfo = fr'{arith.label}(1) = {latexify(arith.start, arith.precision)}, '
        arithInfo += r'\text{ common difference } d = ' + latexify(arith.common, arith.precision)
        geoInfo = fr'{geo.label}(1) = {latexify(geo.start, geo.precision)}, '
        geoInfo += r'\text{ common ratio } r = ' + latexify(geo.common, geo.precision)
    elif info == 2:
        arithInfo, geoInfo = arith.getRecursive(), geo.getRecursive()
    elif info == 3:
        arithInfo, geoInfo = arith.getExplicit(), geo.getExplicit()

    header = ['$n$', f'${arith.label}(n)$', f'${geo.label}(n)$']
    images = [[signify(latexify(arith.subs(jj), arith.precision)), signify(latexify(geo.subs(jj), geo.precision))] for jj in range(1,6)]
    rows = [[f'${jj+1}$', fr'\phantom{brackify(images[jj][0])}', fr'\phantom{brackify(images[jj][1])}'] for jj in range(5)]

    problem = 'Arithmetic sequence: ' + signify(arithInfo) + r'\newline '
    problem += 'Geometric sequence: ' + signify(geoInfo) + r'\newline '
    problem += r'{\bf a. }' + tableGenerator(header, rows)
    problem += r'{\bf b. }' + f'Which value will be greater, ${arith.label}(100)$ or ${geo.label}(100)$? Why?'

    rows = [[f'${jj+1}$', images[jj][0], images[jj][1]] for jj in range(5)]
    answer = r'{\bf a. }' + tableGenerator(header, rows) + answer

    return problem, answer

for jj in range(10):
    problem, answer = Compare_the_Growth_1_9_1(randint(1,5))
    print(problem, r'\\')
    print(answer, r'\\')

Arithmetic sequence: $f(n) = -6\left(n-1\right) + 131$\newline Geometric sequence: $g(n) = 4 \cdot 4^{n - 1}$\newline {\bf a. }{\renewcommand{\arraystretch}{1.5}	\begin{center}
		\begin{tabular}{|c|c|c|}
			\hline
			$n$ & $f(n)$ & $g(n)$ \\
			\hline
			$1$ & \phantom{$131$} & \phantom{$4$} \\
			\hline
			$2$ & \phantom{$125$} & \phantom{$16$} \\
			\hline
			$3$ & \phantom{$119$} & \phantom{$64$} \\
			\hline
			$4$ & \phantom{$113$} & \phantom{$256$} \\
			\hline
			$5$ & \phantom{$107$} & \phantom{$1024$} \\
			\hline
		\end{tabular}
	\end{center}}{\bf b. }Which value will be greater, $f(100)$ or $g(100)$? Why? \\
{\bf a. }{\renewcommand{\arraystretch}{1.5}	\begin{center}
		\begin{tabular}{|c|c|c|}
			\hline
			$n$ & $f(n)$ & $g(n)$ \\
			\hline
			$1$ & $131$ & $4$ \\
			\hline
			$2$ & $125$ & $16$ \\
			\hline
			$3$ & $119$ & $64$ \\
			\hline
			$4$ & $113$ & $256$ \\
			\hline
			$5$ & $107$ & $1024$ \\
			\hline
		\end{tabular}
	\end{center}}{\bf b. }$g(100)>f(100)$: geomet

##1.10.1 Information to Arithmetic(?) Sequence.


### instruction:  write the explicit equation for each geometric sequence.




In [ ]:
for jj in range(10):
    problem, answer = infoToSeq(case=randint(1,4),kind='lin')
    print(problem)
    print(answer)

NameError: ignored

## DOTS (fixed) (can use for pascal's tri)

In [ ]:
def graphical_dot(num):
    num = num + 1
    out = []
    inputted = ""
    out.append(r'\begin{tikzpicture}[main/.style = {draw, circle}]')
    out.append(r'\node[main] (1) {$1$};')
    counter = 2
    num_right=0
    ending=1
    for i in range(counter, num):
        inputted = fr'\node[main] ({counter}) [below left of={ending}] `${counter}$~;'
        counter+=1
        inputted = inputted.replace("`", "{")
        inputted = inputted.replace("~", "}")
        out.append(inputted)
        num_right+=1
        for t in range(num_right):
            inputted = fr'\node[main] ({counter}) [below right of={ending}] `${counter}$~;'
            counter+=1
            ending+=1
            inputted = inputted.replace("`", "{")
            inputted = inputted.replace("~", "}")
            out.append(inputted)

    out.append(r'\end{tikzpicture}')
    return ''.join(out)
print(graphical_dot(9)) # WORKS UNTIL 3

\begin{tikzpicture}[main/.style = {draw, circle}]\node[main] (1) {$1$};\node[main] (2) [below left of=1] {$2$};\node[main] (3) [below right of=1] {$3$};\node[main] (4) [below left of=2] {$4$};\node[main] (5) [below right of=2] {$5$};\node[main] (6) [below right of=3] {$6$};\node[main] (7) [below left of=4] {$7$};\node[main] (8) [below right of=4] {$8$};\node[main] (9) [below right of=5] {$9$};\node[main] (10) [below right of=6] {$10$};\node[main] (11) [below left of=7] {$11$};\node[main] (12) [below right of=7] {$12$};\node[main] (13) [below right of=8] {$13$};\node[main] (14) [below right of=9] {$14$};\node[main] (15) [below right of=10] {$15$};\node[main] (16) [below left of=11] {$16$};\node[main] (17) [below right of=11] {$17$};\node[main] (18) [below right of=12] {$18$};\node[main] (19) [below right of=13] {$19$};\node[main] (20) [below right of=14] {$20$};\node[main] (21) [below right of=15] {$21$};\node[main] (22) [below left of=16] {$22$};\node[main] (23) [below right of=16] {$2

##Archive

In [ ]:
import random
import sympy

def case_1(notation_1, notation_2, start_1, start_2, difference, ratio):
    """1) case 1: same start same diff & r
    problem:
    Arithmetic sequence:  f(1)=3, common difference d=2
    Geometric sequence:  g(1)=3, common ratio r=2
    answer:
    1) find the first five terms:
    2) which grows faster?  f(100)=?  or  g(100)=? """
    arith = fr"{notation_1}(1) = {start_1}, Common Difference: D = {difference}"
    geo = fr"{notation_2}(1) = {start_2}, Common Ratio: R = {ratio}"
    return arith, geo

def case_2(notation_1, notation_2, start_1, start_2, difference, ratio):
    arith = fr"{notation_1}(1) = {start_1}, Common Difference: D = {difference}"
    geo = fr"{notation_2}(1) = {start_2}, Common Ratio: R = {ratio}"
    return arith, geo

def case_3(notation_1, notation_2, start_1, start_2, difference, ratio, variable):

    arith = fr"{start_1} + {difference} * ({variable}-1)"
    geo = fr"{start_2} * ({ratio}) ** {variable}"

    arith = sympy.latex(sympy.sympify(arith, evaluate=False) ,fold_short_frac=False, mode="inline")
    geo = sympy.latex(sympy.sympify(geo, evaluate=False) ,fold_short_frac=False, mode="inline")
    return fr'{notation_1}({variable}) = ' + str(arith), fr'{notation_2}({variable}) = ' + str(geo)


def five_arith_problem_terms_gen(start, ratio):
    function = []
    function.append(str(start))
    for i in range(4):
        function.append(str(int(function[-1]) + int(ratio)))
    return function


def five_geo_problem_terms_gen(start, ratio):
    function = []
    function.append(str(start))
    for i in range(4):
        function.append(str(int(function[-1]) * int(ratio)))
    return function


def solve_eq(notation_1, notation_2, start_1, start_2, difference, ratio, numberoftimes):
    function_arith = int(start_1)
    for i in range(numberoftimes-1):
        function_arith = int(function_arith) + int(difference)
    #function_arith

    function_geo = int(start_2)
    for i in range(numberoftimes-1):
        function_geo = int(function_geo) * int(ratio)

    biggest = 0
    if function_arith == function_geo:
        print("equal")
        biggest = 0
    elif function_arith > function_geo:
        print(function_arith)
        biggest = 1
    elif function_arith < function_geo:
        print(function_geo)
        biggest = 2
    return str(function_arith), str(function_geo), biggest


def Which_Grows_Faster(option_difficulty = "easy", expr="latex"):
    problem=""
    answer=""
    random_num = 0
    variable = random.choice(['x', 'y', 'g', 'b', 'z', 'p', 't', 'q', 'k', 'u', 'r', 'd', 'w', 's', 'h', 'v'])
    if option_difficulty == "easy":
        termtosolve = random.randint(1,10)

        notation_1 = "Z"
        notation_2 = "Y"
        start_1 = random.randint(-5,5)
        start_2 = random.randint(-5,5)
        difference = random.randint(-5,5)
        ratio = random.randint(-5,5)
        random_num = random.randint(1,4)
        if random_num == 1:
            problem = case_1(notation_1, notation_2, start_1, start_2, difference, ratio)
        elif random_num == 2:
            problem = case_2(notation_1, notation_2, start_1, start_2, difference, ratio,)
        else:
            problem = case_3(notation_1, notation_2, start_1, start_2, difference, ratio, variable)
        problem = problem[0] + ", " + problem[1] + fr" \\1) Find the first five terms: \\ 2) Which value will be greater?  {notation_1}({termtosolve}) = ?  or  {notation_2}({termtosolve}) = ?:\\\\"
        arith, geome, biggest = solve_eq(notation_1, notation_2, start_1, start_2, difference, ratio, int(termtosolve))
        #print(arith,geome, biggest)
        if biggest == 0: # if they are equal
            answer = fr"1) Find the first five terms: {notation_1}: `{five_arith_problem_terms_gen(start_1, difference)}~   {notation_2}: `{five_geo_problem_terms_gen(start_2, ratio)}~ \\ 2) Which value will be greater?  They are equal ({notation_1}({termtosolve}) = {arith}  or  {notation_2}({termtosolve}) = {geome})"
        elif biggest == 1: # if arith is bigger
            answer = fr"1) Find the first five terms: {notation_1}: `{five_arith_problem_terms_gen(start_1, difference)}~   {notation_2}: `{five_geo_problem_terms_gen(start_2, ratio)}~ \\ 2) Which value will be greater?  ({notation_1}({termtosolve}) = {arith})"
        elif biggest == 2: #if geo is bigger
            answer = fr"1) Find the first five terms: {notation_1}: `{five_arith_problem_terms_gen(start_1, difference)}~   {notation_2}: `{five_geo_problem_terms_gen(start_2, ratio)}~ \\ 2) Which value will be greater?  ({notation_2}({termtosolve}) = {geome})"

        answer = answer.replace("`", "{")
        answer = answer.replace("~", "}")
        
    return problem, sympy.latex(answer)

for i in range(3):
    a, b = Which_Grows_Faster("easy", "latex")
    print(a,b,r"\\")
"""
easy == start range (-5, 5); D/r -5, 5
medium == start range (-10, 10); D/r -10, 10
hard == start range (-30, 30); D/r -5, 5

"""
"""1.9.1 Which Grows Faster?
section 1
instruction : find the first five terms. Then compare the growth of the arithmetic sequence and the geometric sequence. Which grows faster? When?
"""

In [ ]:
#will clean up code after word problem generation gets finished
import sympy
import random

def dollar_signify(input):
    return("$" + input + "$")


def arith_problem_terms_gen(start, ratio):
    function = []
    function.append(str(start))
    for i in range(random.randint(3,7)):
        function.append(str(int(function[-1]) + int(ratio)))
    return function


def geo_problem_terms_gen(start, ratio):
    function = []
    function.append(str(start))
    for i in range(random.randint(3,7)):
        function.append(str(int(function[-1]) * int(ratio)))
    return function


def recursive_function_formatter(ratio, variable, start, notation):
    return str(sympy.latex(sympy.sympify(fr'{notation} * ({variable} - 1) + {ratio}', evaluate = False), fold_short_frac=False))  + fr", {notation}(1) = {start}"


def explicit_function_formatter(start, variable, ratio):
    return sympy.latex(sympy.sympify(fr'{start} + ({variable} - 1) * {ratio}', evaluate = False), fold_short_frac=False)


def Is_It_Arithmetic_Or_Geometric_Sequence(option_difficulty="easy", expr="latex"):
    variable = random.choice(['x', 'y', 'a', 'b', 'z', 'p', 't'])
    function_start = random.choice(['E', 'K', 'U', 'R', 'D', 'W', 'G', 'H', 'V'])
    problem = ""
    answer = ""
    type_of_sequence = ""
    if option_difficulty == "easy":
        start = random.randint(-10, 10)
        if random.randint(1,2) == 1:
            ratio = random.randint(-5, -1)
        else:
            ratio = random.randint(1, 5)

        if random.randint(1,2) == 1:
            problem = arith_problem_terms_gen(str(start), str(ratio))
            type_of_sequence = "Arithmetic"
        else:
            problem = geo_problem_terms_gen(str(start), str(ratio))
            type_of_sequence = "Geometric"

        for i in range(random.randint(1,len(problem) - 2)):
            problem[random.randint(0,len(problem) - 1)] = r"\_\_\_"
        problem = ', '.join(problem)
        problem = str(problem) + ", ..."
        print(dollar_signify(problem))

        #HOW TO DO PROBLEM AND ANSWER???
        """a) 5,10,15, ____ ,25,30, ____ ,....
        b) Arithmatic or geometric? type_of_sequence
        c) Common Diference or Ratio? ratio
        d) Explicit? Boolean
        e) Recurssive? Boolean
        """

        answer = "Common Difference: " + str(ratio), fr"Recursive Function: {function_start}({variable})" + " = " + recursive_function_formatter(start, variable, ratio, function_start), fr"Explicit Function: {function_start}({variable})" + " = "+ explicit_function_formatter(start, variable, ratio)
        for i in range(len(answer)):
            print(answer[i])

        """1) easy start(-10~10)
        ratio : ( -5~5 except 0)
        a. 3,7,11,15,19,,,___,...
        b. Common Difference: ratio
        c. Recursive Function:A(n) = A(n-1) + d,
        d. Explicit Function:A(n) = A(1) + (n – 1)d
        """

    elif option_difficulty == "medium":
        """2) medium start(-20~20)
        ratio : ( -10 ~ -5 and 5 ~ 10 )
        a. 3,7,11,15,19,,,___,...
        b. Common Difference:
        c. Recursive Function:
        d. Explicit Function:"""
        print()

    elif option_difficulty == "hard":
        """3) hard start(-20~20)
        ratio : ( -30 ~ -10 and 10 ~ 30 )
        a. 3,7,11,15,19,,,___,...
        b. Common Difference:
        c. Recursive Function:
        d. Explicit Function:"""
    return ()
Is_It_Arithmetic_Or_Geometric_Sequence("easy", "latex")

"""1.6.2 Is It Arithmetic Or Geometric Sequence
section1:
instruction: Find the missing values for each arithmetic or geometric sequence. Select whether it has a constant difference or a constant ratio. State the value of the constant difference or ratio. Indicate if the sequence is arithmetic or geometric by selecting the correct answer.

1) Arithmatic sequence
−10to10±5
a) 5,10,15,____,25,30,____,....
b) Arithmatic or geometric? Arith
c) Common Diference or Ratio? +5
d) Explicit?
e) Recurssive?


2) Geometric sequence
−10 10\timediv−5 5
a) 5,10,15,____,25,30,____,....
b) Arithmatic or geometric? Arith
c) Common Diference or Ratio? +5
d) Explicit?
e) Recurssive?


3) Arithmatic word problems
−10 10\timediv−5 5
a) 5,10,15,____,25,30,____,....
b) Arithmatic or geometric? Arith
c) Common Diference or Ratio? +5
d) Explicit?
e) Recurssive?


4) Geometric word problems
−10 10\timediv−5 5
a) 5,10,15,____,25,30,____,....
b) Arithmatic or geometric? Arith
c) Common Diference or Ratio? +5
d) Explicit?
e) Recurssive?"""